In [ ]:
#!/usr/bin/env python3
"""
hybrid_pip_trader_gate_plot.py  (v4)
────────────────────────────────────
• Causal PIP trader  →  suggests positions every bar.
• Causal PnL gate    →  decides (with a 1-bar delay) whether trading is
  enabled (“GREEN”) or disabled (“RED”) based on realised cumulative PnL.
• PIP realised cumulative PnL panel added with PIP markers.

Trading rule
------------
real_pos[t] = suggested_pos[t]   if  gate says GREEN at bar t
            = 0                 otherwise

Key fixes vs previous version
-----------------------------
1. Gate is updated with `cum_real[t-1]` **before** it decides for bar *t*  
   → only a **one-bar** lag, not two.
2. Confidence panel removed.
3. Price shading uses the gate flag, not the suggested position.
4. **PIP realised cumulative PnL** is now plotted in its own panel.
5. **PIP markers** are now added on the PIP pnl plot (fully causal detection).
"""

from pathlib import Path
from typing import List, Optional, Tuple
import numpy as np
import matplotlib.pyplot as plt

# ═════════════ PIP DETECTION HELPERS (causal) ════════════════════════════ #
def _safe_rel_move(px: float, last_ext: float, eps: float = 1e-8) -> float:
    denom = max(abs(last_ext), eps)
    return (px - last_ext) / denom


def find_pips(series: np.ndarray, threshold: float) -> List[Tuple[int, float]]:
    """
    Causal zig‑zag pivot finder. Returns list of (idx, value).
    Marks a pivot as soon as the swing from last extreme exceeds threshold.
    """
    n = len(series)
    pips: List[Tuple[int, float]] = [(0, series[0])]
    last_ext, direction = series[0], None

    for i in range(1, n):
        move = _safe_rel_move(series[i], last_ext)

        if direction is None:
            if abs(move) >= threshold:
                direction = 'up' if move > 0 else 'down'
                last_ext = series[i]
                pips.append((i, series[i]))
        else:
            # extend current extreme
            if direction == 'up' and series[i] > last_ext:
                last_ext = series[i]
            elif direction == 'down' and series[i] < last_ext:
                last_ext = series[i]
            # reversal?
            rev = (_safe_rel_move(series[i], last_ext)
                   if direction == 'down'
                   else -_safe_rel_move(series[i], last_ext))
            if rev >= threshold:
                pips.append((i, last_ext))
                direction = 'down' if direction == 'up' else 'up'
                last_ext = series[i]

    # ensure we mark the final bar
    if pips[-1][0] != n - 1:
        pips.append((n - 1, series[-1]))
    return pips

# ═════════════ STRATEGY PARAMETERS ════════════════════════════════════ #
BEST_N = [
    200,200,400,200,400,400,100,300,100,200,
    100,500,100,300,200,100,200,100,200,300,
    100,200,500,100,400,100,100,400,100,100,
    400,500,300,200,400,500,100,400,100,100,
    100,300,400,500,100,400,500,100,500,300
]
BEST_W = [
    10,10,100,10,100,10,100,25,10,100,
    100,50,25,10,50,25,10,25,50,100,
    100,10,10,50,25,25,10,50,25,100,
    25,10,100,50,25,10,50,100,50,25,
    25,10,100,100,50,50,10,25,25,25
]

POSITION_LIMIT  = 10_000
COMMISSION_RATE = 0.0005
THR_MIN, THR_MAX, THR_STEP = 0.0, 0.700, 0.0005
THR_GRID = np.arange(THR_MIN, THR_MAX + THR_STEP, THR_STEP)

GATE_THR = 0.10  # 10% swing on cum-PnL flips gate
EPS = 1e-8

# ═════════════ LOW-LEVEL HELPERS / TRADER / GATE ═══════════════════════ #
def safe_rel_move(a: float, b: float, eps: float = EPS) -> float:
    return (a - b) / max(abs(b), eps)


def pip_update(px: float, last_ext: float, direction: Optional[int], thr: float) -> Tuple[int, float]:
    if direction == 0 or direction is None:
        mv = (px - last_ext) / last_ext
        if abs(mv) >= thr:
            return (1 if mv > 0 else -1), px
        return 0, last_ext
    if direction == 1:
        if px > last_ext:
            return 1, px
        if (last_ext - px) / last_ext >= thr:
            return -1, px
        return 1, last_ext
    if px < last_ext:
        return -1, px
    if (px - last_ext) / last_ext >= thr:
        return 1, px
    return -1, last_ext


class CausalPIPTrader:
    class _S:
        __slots__ = ("n", "w", "next_calib", "thr", "dir", "last_ext")
        def __init__(self, n: int, w: int):
            self.n, self.w = n, w
            self.next_calib = n
            self.thr = None
            self.dir = 0
            self.last_ext = None

    def __init__(self):
        self._st = [self._S(n, w) for n, w in zip(BEST_N, BEST_W)]

    def _best_thr(self, window: np.ndarray) -> float:
        best_thr, best_pnl = THR_GRID[0], -np.inf
        for thr in THR_GRID:
            dvec = np.zeros(len(window), np.int8)
            le, d = window[0], 0
            for k in range(1, len(window)):
                d, le = pip_update(window[k], le, d, thr)
                dvec[k] = d
            pnl = np.sum(dvec[:-1].astype(np.int32) * POSITION_LIMIT * np.diff(window))
            if pnl > best_pnl:
                best_pnl, best_thr = pnl, thr
        return best_thr

    def Alg(self, prc_so_far: np.ndarray) -> np.ndarray:
        nInst, t_seen = prc_so_far.shape
        pos = np.zeros(nInst, np.int32)
        for i in range(nInst):
            st, px = self._st[i], prc_so_far[i, -1]
            if st.last_ext is None:
                st.last_ext = px
            if t_seen >= st.next_calib:
                st.thr = self._best_thr(prc_so_far[i, t_seen - st.n:t_seen])
                st.next_calib += st.w
            if st.thr is None:
                continue
            new_dir, new_ext = pip_update(px, st.last_ext, st.dir, st.thr)
            st.dir, st.last_ext = new_dir, new_ext
            pos[i] = new_dir * POSITION_LIMIT
        return pos


class GateState:
    __slots__ = ("direction", "last_ext")
    def __init__(self):
        self.direction: Optional[str] = None
        self.last_ext = 0.0

    def update(self, pnl_now: float) -> None:
        mv = safe_rel_move(pnl_now, self.last_ext)
        if self.direction is None:
            if abs(mv) >= GATE_THR:
                self.direction = 'up' if mv > 0 else 'down'
                self.last_ext = pnl_now
            return
        if self.direction == 'up':
            if pnl_now > self.last_ext:
                self.last_ext = pnl_now
            elif safe_rel_move(pnl_now, self.last_ext) <= -GATE_THR:
                self.direction, self.last_ext = 'down', pnl_now
        else:
            if pnl_now < self.last_ext:
                self.last_ext = pnl_now
            elif safe_rel_move(pnl_now, self.last_ext) >= GATE_THR:
                self.direction, self.last_ext = 'up', pnl_now

    @property
    def allow(self) -> bool:
        return self.direction == 'up'


# ═════════════ MAIN LOOP: SIMULATION ═══════════════════════════════════ #
prices = np.loadtxt(Path("prices.txt"))
T, nInst = prices.shape
trader = CausalPIPTrader()
gates = [GateState() for _ in range(nInst)]

pos_sugg = np.zeros((T, nInst), int)
pos_real = np.zeros((T, nInst), int)
allow_flag = np.zeros((T, nInst), bool)

pnl_sugg = np.zeros((T, nInst), float)
cum_sugg = np.zeros((T, nInst), float)

pnl_real = np.zeros((T, nInst), float)
cum_real = np.zeros((T, nInst), float)

prev_sugg = np.zeros(nInst, int)
prev_real = np.zeros(nInst, int)

for t in range(T):
    if t > 0:
        for i in range(nInst):
            gates[i].update(cum_real[t-1, i])

    allow_flag[t] = np.array([g.allow for g in gates], bool)
    sugg = trader.Alg(prices[:t+1].T)
    pos_sugg[t] = sugg
    pos_real[t] = np.where(allow_flag[t], sugg, 0)

    if t > 0:
        diff = prices[t] - prices[t-1]
        trade_s = np.abs(sugg - prev_sugg) * prices[t]
        pnl_sugg[t] = prev_sugg * diff - COMMISSION_RATE * trade_s
        cum_sugg[t] = cum_sugg[t-1] + pnl_sugg[t]

        trade_r = np.abs(pos_real[t] - prev_real) * prices[t]
        pnl_real[t] = prev_real * diff - COMMISSION_RATE * trade_r
        cum_real[t] = cum_real[t-1] + pnl_real[t]

    prev_sugg, prev_real = sugg, pos_real[t]

print("Simulation complete.")


# ═════════════ PLOTS WITH PIP MARKERS ═══════════════════════════════════ #
# choose pip detection threshold (e.g. 0.05 for 5% moves)
pip_threshold = 0.15

times = np.arange(T)
for i in range(nInst):
    pivots = find_pips(cum_sugg[:, i], pip_threshold)
    idxs, vals = zip(*pivots)

    fig, (ax1, ax2, ax3) = plt.subplots(
        3, 1, sharex=True, figsize=(10, 12),
        gridspec_kw={'height_ratios': [1, 1, 1]}
    )

    # Price + gate shading
    ax1.plot(times, prices[:, i], color='black', lw=1)
    ax1.fill_between(
        times,
        prices[:, i].min(), prices[:, i].max(),
        where=allow_flag[:, i], color='green', alpha=0.18, step='mid'
    )
    ax1.fill_between(
        times,
        prices[:, i].min(), prices[:, i].max(),
        where=~allow_flag[:, i], color='red', alpha=0.18, step='mid'
    )
    ax1.set_ylabel('Price')

    # Gated PnL
    ax2.plot(
        times, cum_real[:, i], lw=1.2, color='black', label='Realised PnL (gated)'
    )
    ax2.axhline(0, lw=0.5)
    ax2.set_ylabel('Gated PnL')
    ax2.legend(frameon=False, fontsize='small')

    # PIP PnL + markers
    ax3.plot(
        times, cum_sugg[:, i], lw=1.2, ls='--', label='PIP Realised Cumulative PnL'
    )
    ax3.scatter(idxs, vals, s=50, color='blue', label='PIPs')
    ax3.axhline(0, lw=0.5)
    ax3.set_ylabel('PIP PnL')
    ax3.set_xlabel('Timestep')
    ax3.legend(frameon=False, fontsize='small')

    fig.suptitle(f'Instrument {i:02d} — Gate‑Aware & PIP PnL with PIPs')
    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    plt.show()


In [ ]:
# %% [markdown]
# # Full “PnL‑gated causal PIP trader” notebook helper
#
# • Includes the **complete** trader model (`CausalPIPTraderPNLGate`).  
# • Streams prices bar‑by‑bar, books commission exactly like `eval.py`.  
# • Tracks underlying daily PnL, cumulative realised PnL, cash & positions.  
# • Plots – per instrument – three panels:
#     1. daily PnL  
#     2. cumulative PnL  
#     3. price with buy / sell markers
#
# Drop the whole cell into Jupyter / JupyterLab and run.

# %%
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# ─────────────────────── trader model ────────────────────────────────────
POSITION_LIMIT        = 10_000
COMM_RATE             = 0.0005          # 50 bps
LOSS_LIMIT_FRAC       = 0.01            # 1 % of the start price
THR_MIN, THR_MAX, THR_STEP = 0.0, 0.700, 0.0005
THR_GRID              = np.arange(THR_MIN, THR_MAX + THR_STEP, THR_STEP)

BEST_N = [
    200, 200, 400, 200, 400, 400, 100, 300, 100, 200,
    100, 500, 100, 300, 200, 100, 200, 100, 200, 300,
    100, 200, 500, 100, 400, 100, 100, 400, 100, 100,
    400, 500, 300, 200, 400, 500, 100, 400, 100, 100,
    100, 300, 400, 500, 100, 400, 500, 100, 500, 300,
]
BEST_W = [
     10,  10, 100,  10, 100,  10, 100,  25,  10, 100,
    100,  50,  25,  10,  50,  25,  10,  25,  50, 100,
    100,  10,  10,  50,  25,  25,  10,  50,  25, 100,
     25,  10, 100,  50,  25,  10,  50, 100,  50,  25,
     25,  10, 100, 100,  50,  50,  10,  25,  25,  25,
]

def causal_update(px: float,
                  last_extreme: float,
                  direction: int | None,
                  thr: float) -> tuple[int, float]:
    if direction in (0, None):
        move = (px - last_extreme) / last_extreme
        if abs(move) >= thr:
            return (1 if move > 0 else -1), px
        return 0, last_extreme

    if direction == 1:                         # up‑swing
        if px > last_extreme:
            return 1, px
        if (last_extreme - px) / last_extreme >= thr:
            return -1, px
        return 1, last_extreme

    # direction == −1 (down‑swing)
    if px < last_extreme:
        return -1, px
    if (px - last_extreme) / last_extreme >= thr:
        return 1, px
    return -1, last_extreme


def best_threshold(window: np.ndarray) -> float:
    """Overflow‑safe grid search for the causal‑PIP threshold."""
    best_thr, best_pnl = THR_GRID[0], -np.inf
    for thr in THR_GRID:
        dir_vec = np.zeros(len(window), np.int8)
        le, d = window[0], 0
        for k in range(1, len(window)):
            d, le = causal_update(window[k], le, d, thr)
            dir_vec[k] = d
        pnl = np.sum(dir_vec[:-1].astype(np.int32) *
                     POSITION_LIMIT *
                     np.diff(window))
        if pnl > best_pnl:
            best_pnl, best_thr = pnl, thr
    return best_thr


class CausalPIPTraderPNLGate:
    """
    Rolling‑window causal PIP trader with per‑instrument PnL gating.

    • Disables an instrument when its account value drops below −1 % of
      the start price; re‑enables once value rises above zero.
    • Commission 0.0005 on traded notional.
    """
    class _State:
        __slots__ = ("n", "w", "next_cal", "thr", "dir",
                     "le", "pos", "cash", "enabled",
                     "start_px", "prev_px")

        def __init__(self, n: int, w: int):
            self.n, self.w    = n, w
            self.next_cal     = n
            self.thr          = None
            self.dir          = 0
            self.le           = None
            self.pos          = 0
            self.cash         = 0.0
            self.enabled      = True
            self.start_px     = None
            self.prev_px      = None

    def __init__(self):
        self._states = [self._State(n, w) for n, w in zip(BEST_N, BEST_W)]

    # ------------------------------------------------------------------
    def Alg(self, prc_so_far: np.ndarray) -> np.ndarray:
        n_inst, t_seen = prc_so_far.shape
        if n_inst != 50:
            raise ValueError(f"Expected 50 instruments, got {n_inst}.")

        out = np.zeros(n_inst, np.int32)

        for i in range(n_inst):
            st  = self._states[i]
            px  = prc_so_far[i, -1]

            # first observation
            if st.start_px is None:
                st.start_px = px
                st.le       = px
                st.prev_px  = px

            # update hidden account value & gate
            value = st.cash + st.pos * px
            if value <= -LOSS_LIMIT_FRAC * st.start_px:
                st.enabled = False
            elif value > 0:
                st.enabled = True

            # (re‑)calibrate threshold
            if t_seen >= st.next_cal:
                win = prc_so_far[i, t_seen - st.n:t_seen]
                st.thr = best_threshold(win)
                st.next_cal += st.w

            # decide desired position
            desired_dir = 0
            if st.enabled and st.thr is not None:
                st.dir, st.le = causal_update(px, st.le, st.dir, st.thr)
                desired_dir   = st.dir

            desired_pos = desired_dir * POSITION_LIMIT

            # execute trade
            delta_pos = desired_pos - st.pos
            if delta_pos != 0:
                st.cash -= px * delta_pos + abs(px * delta_pos) * COMM_RATE
                st.pos   = desired_pos

            out[i]   = st.pos
            st.prev_px = px

        return out

# ─────────────────── simulation & plotting ───────────────────────────────
def simulate_stream(prc: np.ndarray,
                    comm_rate: float = COMM_RATE,
                    dlr_pos_limit: float = POSITION_LIMIT):
    """
    Stream the price matrix through the trader bar‑by‑bar.

    Returns a dict with 'pos', 'cash', 'value', 'daily_pnl'.
    """
    n_inst, T = prc.shape
    trader = CausalPIPTraderPNLGate()

    pos   = np.zeros((n_inst, T),  np.int32)
    cash  = np.zeros((n_inst, T),  np.float64)
    value = np.zeros((n_inst, T),  np.float64)

    for t in range(T):
        cur_px = prc[:, t]
        new_pos = trader.Alg(prc[:, :t + 1])

        # enforce $ limit like eval.py
        pos_limits = (dlr_pos_limit / cur_px).astype(int)
        new_pos = np.clip(new_pos, -pos_limits, pos_limits)

        if t == 0:
            delta_pos = new_pos
            traded_notional = cur_px * abs(delta_pos)
            cash[:, t] = -cur_px * delta_pos - traded_notional * comm_rate
        else:
            delta_pos = new_pos - pos[:, t - 1]
            traded_notional = cur_px * abs(delta_pos)
            cash[:, t] = cash[:, t - 1] - cur_px * delta_pos - traded_notional * comm_rate

        pos[:, t] = new_pos
        value[:, t] = cash[:, t] + pos[:, t] * cur_px

    daily_pnl = np.vstack([
        np.full(n_inst, np.nan),
        np.diff(value, axis=1).T
    ]).T

    return dict(pos=pos, cash=cash, value=value, daily_pnl=daily_pnl)


def plot_instrument(prc: np.ndarray,
                    sim: dict,
                    inst: int,
                    show: bool = True):
    """Three‑panel plot for one instrument."""
    px   = prc[inst]
    pnl  = sim["daily_pnl"][inst]
    cum  = sim["value"][inst] - sim["value"][inst, 0]
    pos  = sim["pos"][inst]

    trades = np.where(np.diff(np.hstack([[0], pos])) != 0)[0]
    buys  = trades[pos[trades] > 0]
    sells = trades[pos[trades] < 0]

    fig, axs = plt.subplots(3, 1, figsize=(12, 9), sharex=True,
                            gridspec_kw={'height_ratios': [1, 1, 2]})
    t = np.arange(px.size)

    # daily PnL
    axs[0].plot(t, pnl, lw=0.8)
    axs[0].axhline(0, color="k", lw=0.5)
    axs[0].set_ylabel("daily PnL ($)")
    axs[0].set_title(f"Instrument {inst}")

    # cumulative PnL
    axs[1].plot(t, cum, lw=1.0)
    axs[1].axhline(0, color="k", lw=0.5)
    axs[1].set_ylabel("cum PnL ($)")

    # price with trades
    axs[2].plot(t, px, lw=1.0)
    axs[2].scatter(buys,  px[buys],  marker="^", s=60, label="buy")
    axs[2].scatter(sells, px[sells], marker="v", s=60, label="sell")
    axs[2].set_ylabel("price")
    axs[2].legend()
    axs[2].set_xlabel("bar")

    plt.tight_layout()
    if show:
        plt.show()
    return fig


# ─────────────────── data loader / demo ────────────────────────────────
def load_prices(fname: str = "prices.txt"):
    """Load 50×T price matrix (rows = instruments). If not found, create mock."""
    try:
        m = np.loadtxt(fname)
        if m.shape[0] != 50:                     # maybe it's T×50
            m = m.T
        print(f"Loaded {fname}  →  {m.shape}")
        return m, True
    except FileNotFoundError:
        # mock geometric random‑walk (3 instruments, 600 bars)
        n_inst, T = 3, 600
        rng = np.random.default_rng(42)
        log_ret = 0.0003 + 0.01 * rng.standard_normal((n_inst, T))
        mock = 100 * np.exp(np.cumsum(log_ret, axis=1))
        print("No prices.txt – using synthetic random‑walk.")
        return mock, False


if __name__ == "__main__":
    PX, real = load_prices()
    sim = simulate_stream(PX)

    # Plot the first 3 instruments (or however many exist)
    for i in range(min(3, PX.shape[0])):
        plot_instrument(PX, sim, i)


In [ ]:
#!/usr/bin/env python3
"""
dynamic_pip_trader_pnl_gate.py
──────────────────────────────
Rolling‑window **causal PIP** trader with a *dynamic PnL gate*:

    • The *underlying* PIP strategy (‑POSITION_LIMIT…+POSITION_LIMIT
      shares) is **always** simulated, including commissions
      (0.0005 × $‑notional on every turn).

    • For each instrument we keep the running
          net_pnl = underlying_pnl − commissions
      from t = 0.

    • If `net_pnl > 0` the instrument is **enabled** and the
      *real* position matches the underlying suggestion.

    • Otherwise the instrument is **suppressed** — we immediately trade
      to flat and stay flat (though we keep *simulating* the
      underlying strategy so net_pnl can swing back above 0 and
      re‑enable trading).

Everything else (N/W calibration cadence, threshold grid, position
sizing) is identical to the previous causal‑PIP implementation.

Exported class
∴  `CausalPIPTraderPNLGate`
"""
from typing import List, Optional, Tuple
import numpy as np
from Model.standard_template import Trader, export

# ───────── hyper‑parameters ──────────────────────────────────────────────
POSITION_LIMIT              = 10_000
COMM_RATE                   = 0.0005                 # $ commission fraction
THR_MIN, THR_MAX, THR_STEP  = 0.0, 0.700, 0.0005
THR_GRID                    = np.arange(THR_MIN, THR_MAX + THR_STEP, THR_STEP)

BEST_N: List[int] = [
    200, 200, 400, 200, 400, 400, 100, 300, 100, 200,
    100, 500, 100, 300, 200, 100, 200, 100, 200, 300,
    100, 200, 500, 100, 400, 100, 100, 400, 100, 100,
    400, 500, 300, 200, 400, 500, 100, 400, 100, 100,
    100, 300, 400, 500, 100, 400, 500, 100, 500, 300,
]
BEST_W: List[int] = [
     10,  10, 100,  10, 100,  10, 100,  25,  10, 100,
    100,  50,  25,  10,  50,  25,  10,  25,  50, 100,
    100,  10,  10,  50,  25,  25,  10,  50,  25, 100,
     25,  10, 100,  50,  25,  10,  50, 100,  50,  25,
     25,  10, 100, 100,  50,  50,  10,  25,  25,  25,
]

# ───────── helper functions ──────────────────────────────────────────────
def causal_update(px: float,
                  last_extreme: float,
                  direction: Optional[int],
                  thr: float) -> Tuple[int, float]:
    if direction in (0, None):
        move = (px - last_extreme) / last_extreme
        if abs(move) >= thr:
            return (1 if move > 0 else -1), px
        return 0, last_extreme

    if direction == 1:                               # up‑swing
        if px > last_extreme:
            return 1, px
        if (last_extreme - px) / last_extreme >= thr:
            return -1, px
        return 1, last_extreme

    # direction == −1 (down‑swing)
    if px < last_extreme:
        return -1, px
    if (px - last_extreme) / last_extreme >= thr:
        return 1, px
    return -1, last_extreme


def best_threshold(window: np.ndarray) -> float:
    """Overflow‑safe grid search for the causal‑PIP threshold."""
    best_thr, best_pnl = THR_GRID[0], -np.inf
    for thr in THR_GRID:
        dir_vec = np.zeros(len(window), np.int8)
        le, d   = window[0], 0
        for k in range(1, len(window)):
            d, le = causal_update(window[k], le, d, thr)
            dir_vec[k] = d
        pnl = np.sum(dir_vec[:-1].astype(np.int32) *
                     POSITION_LIMIT *
                     np.diff(window))
        if pnl > best_pnl:
            best_pnl, best_thr = pnl, thr
    return best_thr


# ───────── trader class ──────────────────────────────────────────────────
class CausalPIPTraderPNLGate(Trader):
    """
    Causal‑PIP trader that *only* trades an instrument while its
    underlying simulated PnL (after commissions) is positive.
    """
    class _State:
        __slots__ = ("n", "w", "next_cal", "thr",
                     "dir", "prev_dir", "le",
                     "hypo_pnl", "prev_px",
                     "enabled",
                     "pos", "cash")

        def __init__(self, n: int, w: int):
            self.n, self.w    = n, w
            self.next_cal     = n
            self.thr          = None
            self.dir          = 0
            self.prev_dir     = 0
            self.le           = None
            self.hypo_pnl     = 0.0
            self.prev_px      = None
            self.enabled      = True
            self.pos          = 0           # real position (shares)
            self.cash         = 0.0         # real cash

    # ────────────────────────────────────────────────────────────────
    def __init__(self):
        super().__init__()
        self._states = [self._State(n, w) for n, w in zip(BEST_N, BEST_W)]
        print("CausalPIPTraderPNLGate ready.")

    # ----------------------------------------------------------------
    @export
    def Alg(self, prcSoFar: np.ndarray) -> np.ndarray:
        n_inst, t_seen = prcSoFar.shape
        if n_inst != 50:
            raise ValueError(f"Expected 50 instruments, got {n_inst}.")

        positions = np.zeros(n_inst, np.int32)

        for i in range(n_inst):
            st  = self._states[i]
            px  = prcSoFar[i, -1]

            # ─── first observation ──────────────────────────────────
            if st.prev_px is None:
                st.prev_px  = px
                st.le       = px

            # ─── (re‑)calibrate threshold every W bars ─────────────
            if t_seen >= st.next_cal:
                win = prcSoFar[i, t_seen - st.n : t_seen]
                st.thr = best_threshold(win)
                st.next_cal += st.w
                # print(f"[t={t_seen:4d}] Inst {i:2d} thr={st.thr:.4f}")

            # ─── update underlying direction (always) ───────────────
            if st.thr is not None:
                st.dir, st.le = causal_update(px, st.le, st.dir, st.thr)

            # ─── update *hypothetical* pnl & commissions ────────────
            price_move = px - st.prev_px
            st.hypo_pnl += st.prev_dir * POSITION_LIMIT * price_move

            if st.dir != st.prev_dir:
                delta_sh = (st.dir - st.prev_dir) * POSITION_LIMIT
                st.hypo_pnl -= abs(delta_sh) * px * COMM_RATE

            st.prev_dir = st.dir
            st.prev_px  = px

            # ─── gate decision ──────────────────────────────────────
            st.enabled = st.hypo_pnl > 0.0

            # ─── desired real position ──────────────────────────────
            desired_pos = (st.dir * POSITION_LIMIT) if st.enabled else 0
            delta_pos   = desired_pos - st.pos

            if delta_pos != 0:
                st.cash -= px * delta_pos + abs(px * delta_pos) * COMM_RATE
                st.pos   = desired_pos

            positions[i] = st.pos

        return positions


__all__ = ["CausalPIPTraderPNLGate"]


In [ ]:
#!/usr/bin/env python3
"""
analyse_blacklist_vs_others.py
──────────────────────────────
Explores similarities / differences between the seven under‑performing
instruments and the rest of the universe.

Assumptions
-----------
• `prices.txt` is a whitespace‑delimited file with shape (50, T)
  –  Row i     = close prices for instrument i
  –  Columns   = chronological bars (oldest → newest)

Outputs
-------
• Feature DataFrame (one row per instrument)
• Summary table contrasting BLACK_LIST vs. non‑blacklist
• Welch t‑test p‑values for each feature
• Correlation heatmap, PCA 2‑D scatter + clusters, hierarchical dendrogram

Dependencies
------------
numpy, pandas, scipy, scikit‑learn, matplotlib, seaborn (optional but nicer plots)
"""
from pathlib import Path
from typing import Tuple, Dict, List

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import skew, kurtosis, ttest_ind
from scipy.cluster.hierarchy import linkage, dendrogram, fcluster
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# ──────────────────── 1. settings ────────────────────────────────────────── #
BLACK_LIST: set[int] = {48, 45, 35, 22, 5, 23, 49}
PRICE_FILE           = "prices.txt"      # change if needed
ANNUALISATION_FACTOR = 252              # daily data → annual metrics

# ──────────────────── 2. helpers ────────────────────────────────────────── #
def load_prices(fname: str = PRICE_FILE) -> np.ndarray:
    for d in (Path.cwd(), *Path.cwd().parents):
        f = d / fname
        if f.exists():
            print(f"Loaded prices from {f}")
            return np.loadtxt(f)    # (n_inst, T)
    raise FileNotFoundError(fname)

def realised_vol(returns: np.ndarray) -> float:
    return returns.std(ddof=0) * np.sqrt(ANNUALISATION_FACTOR)

def sharpe_ratio(returns: np.ndarray) -> float:
    # risk‑free ~ 0 for short look‑backs; safish
    sigma = returns.std(ddof=0)
    return 0.0 if sigma == 0 else returns.mean() / sigma * np.sqrt(ANNUALISATION_FACTOR)

def max_drawdown(prices: np.ndarray) -> float:
    cummax = np.maximum.accumulate(prices)
    drawdown = (prices - cummax) / cummax
    return drawdown.min()     # most negative

def approx_atr(prices: np.ndarray, window: int = 14) -> float:
    tr = np.abs(np.diff(prices))                 # | close_t − close_{t-1} |
    if tr.size < window:
        return tr.mean()
    # simple moving average of TR as ATR proxy
    atr = pd.Series(tr).rolling(window, min_periods=1).mean().iloc[-1]
    return float(atr)

def beta_to_market(inst_ret: np.ndarray, mkt_ret: np.ndarray) -> float:
    cov = np.cov(inst_ret, mkt_ret, ddof=0)[0, 1]
    var = mkt_ret.var(ddof=0)
    return 0.0 if var == 0 else cov / var

# ──────────────────── 3. main analysis ──────────────────────────────────── #
def build_feature_df(px: np.ndarray) -> pd.DataFrame:
    n_inst, T = px.shape
    ret = np.diff(px) / px[:, :-1]     # one‑bar arithmetic returns
    market_ret = ret.mean(axis=0)

    feats: Dict[str, List[float]] = {k: [] for k in [
        "mean_ret", "vol", "sharpe", "skew", "kurtosis",
        "max_dd", "atr14", "beta", "autocorr1"
    ]}

    for i in range(n_inst):
        r = ret[i]
        p = px[i]

        feats["mean_ret"].append(r.mean() * ANNUALISATION_FACTOR)
        feats["vol"].append(realised_vol(r))
        feats["sharpe"].append(sharpe_ratio(r))
        feats["skew"].append(skew(r, bias=False) if r.size > 1 else np.nan)
        feats["kurtosis"].append(kurtosis(r, bias=False, fisher=True) if r.size > 1 else np.nan)
        feats["max_dd"].append(max_drawdown(p))
        feats["atr14"].append(approx_atr(p, 14))
        feats["beta"].append(beta_to_market(r, market_ret))
        feats["autocorr1"].append(pd.Series(r).autocorr(lag=1))

    df = pd.DataFrame(feats)
    df.index.name = "instrument"
    return df

def compare_groups(df: pd.DataFrame, blacklist: set[int]) -> Tuple[pd.DataFrame, pd.DataFrame]:
    mask = df.index.isin(blacklist)
    grp_bad  = df[mask]
    grp_good = df[~mask]

    summary = pd.concat({
        "black_list_mean":   grp_bad.mean(),
        "others_mean":       grp_good.mean(),
        "difference":        grp_bad.mean() - grp_good.mean()
    }, axis=1)

    pvals = {
        f: ttest_ind(grp_bad[f], grp_good[f], equal_var=False, nan_policy="omit").pvalue
        for f in df.columns
    }
    pval_df = pd.Series(pvals, name="Welch p‑value").to_frame()

    return summary, pval_df

def correlation_heatmap(ret: np.ndarray):
    corr = np.corrcoef(ret)
    mask = np.triu(np.ones_like(corr, bool))
    plt.figure(figsize=(10,8))
    sns.heatmap(corr, mask=mask, cmap="coolwarm", center=0, vmin=-1, vmax=1,
                cbar_kws={"shrink":0.8}, square=True)
    plt.title("Pairwise return correlations")
    plt.tight_layout()
    plt.show()

def pca_and_cluster(df: pd.DataFrame, blacklist: set[int]):
    X = StandardScaler().fit_transform(df.values)
    pca = PCA(n_components=2, random_state=0)
    comp = pca.fit_transform(X)

    # Agglomerative clustering on PCs (Ward)
    Z = linkage(comp, method="ward")
    clusters = fcluster(Z, t=4, criterion="maxclust")   # 4 clusters heuristic

    # 2‑D scatter
    plt.figure(figsize=(8,6))
    palette = sns.color_palette("tab10", clusters.max())
    for i, idx in enumerate(df.index):
        lbl = "BL" if idx in blacklist else "OK"
        plt.scatter(comp[i,0], comp[i,1],
                    marker="^" if idx in blacklist else "o",
                    s=80,
                    color=palette[clusters[i]-1],
                    label=lbl if (lbl not in plt.gca().get_legend_handles_labels()[1]) else "")
    plt.xlabel("PC‑1")
    plt.ylabel("PC‑2")
    plt.title("PCA scatter with Ward clusters")
    plt.legend()
    plt.tight_layout()
    plt.show()

    # dendrogram
    plt.figure(figsize=(12,4))
    dendrogram(Z, labels=df.index, leaf_rotation=90)
    plt.title("Hierarchical clustering dendrogram (Ward, Euclidean on PCs)")
    plt.tight_layout()
    plt.show()

# ──────────────────── 4. run everything ────────────────────────────────── #
def main():
    prices = load_prices()                 # shape (50, T)
    df_feat = build_feature_df(prices)
    print("\n=== Per‑instrument feature matrix ===")
    print(df_feat.round(4).to_string())

    summary, pvals = compare_groups(df_feat, BLACK_LIST)
    print("\n=== Group comparison: means & differences ===")
    print(summary.round(4).to_string())
    print("\n=== Welch unequal‑var t‑test p‑values ===")
    print(pvals.round(4).to_string())

    # Correlation heatmap
    correlation_heatmap(np.diff(prices) / prices[:, :-1])

    # PCA scatter + clustering + dendrogram
    pca_and_cluster(df_feat, BLACK_LIST)

if __name__ == "__main__":
    main()


In [ ]:
#!/usr/bin/env python3
"""
analyse_blacklist_vs_others.py
──────────────────────────────
Explores similarities / differences between the seven under‑performing
instruments and the rest of the universe—given that your `prices.txt`
is 1 000 rows × 50 columns, each row a time (earliest at top, most recent
at bottom) and each column an instrument (0–49).

Outputs
-------
• Per‑instrument feature DataFrame (50 × 9)
• Summary table: BLACK_LIST vs. others, plus Welch’s t‑test p‑values
• 50×50 instrument return correlation heatmap
• PCA 2‑D scatter (black ▲ for BLACK_LIST, colored ● for others)
• Hierarchical dendrogram with leaves labeled 0–49
"""
from pathlib import Path
from typing import Tuple, Dict, List

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import skew, kurtosis, ttest_ind
from scipy.cluster.hierarchy import linkage, dendrogram, fcluster
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# ──────────────────── 1. settings ────────────────────────────────────────── #
BLACK_LIST: set[int] = {48, 45, 35, 22, 5, 23, 49}
PRICE_FILE           = "prices.txt"
ANNUALISATION_FACTOR = 252  # to annualize daily statistics

# ──────────────────── 2. data loading ─────────────────────────────────────── #
def load_prices(fname: str = PRICE_FILE) -> np.ndarray:
    """
    Load a whitespace‑delimited file of shape (1000, 50), where each
    row is a timestamp (oldest→newest) and each column is an instrument.
    Returns a (50, 1000) array px so that px[i, t] is price of inst i at bar t.
    """
    p = Path(fname)
    if not p.exists():
        raise FileNotFoundError(f"Expected {fname} in CWD or parent directories.")
    raw = np.loadtxt(p)  # shape (T=1000, n_inst=50)
    if raw.ndim != 2 or raw.shape[1] != 50:
        raise ValueError(f"Expected file with 50 columns, got shape {raw.shape}")
    # transpose so rows=instruments, cols=time
    px = raw.T
    print(f"Loaded prices: instruments={px.shape[0]}, timesteps={px.shape[1]}")
    return px  # (50, 1000)

# ──────────────────── 3. feature helpers ─────────────────────────────────── #
def realised_vol(returns: np.ndarray) -> float:
    return returns.std(ddof=0) * np.sqrt(ANNUALISATION_FACTOR)

def sharpe_ratio(returns: np.ndarray) -> float:
    sigma = returns.std(ddof=0)
    return 0.0 if sigma == 0 else returns.mean() / sigma * np.sqrt(ANNUALISATION_FACTOR)

def max_drawdown(prices: np.ndarray) -> float:
    cummax = np.maximum.accumulate(prices)
    return ((prices - cummax) / cummax).min()

def approx_atr(prices: np.ndarray, window: int = 14) -> float:
    tr = np.abs(np.diff(prices))
    if tr.size < window:
        return tr.mean()
    return float(pd.Series(tr).rolling(window, min_periods=1).mean().iloc[-1])

def beta_to_market(inst_ret: np.ndarray, mkt_ret: np.ndarray) -> float:
    cov = np.cov(inst_ret, mkt_ret, ddof=0)[0, 1]
    var = mkt_ret.var(ddof=0)
    return 0.0 if var == 0 else cov / var

# ──────────────────── 4. build feature DataFrame ────────────────────────── #
def build_feature_df(px: np.ndarray) -> pd.DataFrame:
    """
    px: (50, 1000) price matrix
    returns: DataFrame indexed 0–49 with cols:
      mean_ret, vol, sharpe, skew, kurtosis,
      max_dd, atr14, beta, autocorr1
    """
    n_inst, T = px.shape
    # one-bar returns shape (50, 999)
    ret = np.diff(px, axis=1) / px[:, :-1]
    # average return across instruments = "market"
    market_ret = ret.mean(axis=0)

    feats: Dict[str, List[float]] = {k: [] for k in [
        "mean_ret", "vol", "sharpe", "skew", "kurtosis",
        "max_dd", "atr14", "beta", "autocorr1"
    ]}

    for i in range(n_inst):
        r = ret[i]
        p = px[i]
        feats["mean_ret"].append(r.mean() * ANNUALISATION_FACTOR)
        feats["vol"].append(realised_vol(r))
        feats["sharpe"].append(sharpe_ratio(r))
        feats["skew"].append(skew(r, bias=False) if r.size > 1 else np.nan)
        feats["kurtosis"].append(kurtosis(r, bias=False, fisher=True) if r.size > 1 else np.nan)
        feats["max_dd"].append(max_drawdown(p))
        feats["atr14"].append(approx_atr(p, 14))
        feats["beta"].append(beta_to_market(r, market_ret))
        feats["autocorr1"].append(pd.Series(r).autocorr(lag=1))

    df = pd.DataFrame(feats)
    df.index.name = "instrument"
    return df

# ──────────────────── 5. compare BLACK_LIST vs. others ────────────────────── #
def compare_groups(df: pd.DataFrame, blacklist: set[int]) -> Tuple[pd.DataFrame, pd.DataFrame]:
    mask     = df.index.isin(blacklist)
    grp_bad  = df[mask]
    grp_good = df[~mask]

    summary = pd.concat({
        "black_list_mean": grp_bad.mean(),
        "others_mean":     grp_good.mean(),
        "difference":      grp_bad.mean() - grp_good.mean()
    }, axis=1)

    pvals = {
        feat: ttest_ind(grp_bad[feat], grp_good[feat], equal_var=False, nan_policy="omit").pvalue
        for feat in df.columns
    }
    pval_df = pd.Series(pvals, name="Welch p‑value").to_frame()
    return summary, pval_df

# ──────────────────── 6. 50×50 correlation heatmap ────────────────────────── #
def correlation_heatmap(px: np.ndarray):
    """
    px: (50, 1000) price matrix → compute returns → 50×50 corr
    """
    ret = np.diff(px, axis=1) / px[:, :-1]       # (50, 999)
    df_ret = pd.DataFrame(ret.T, columns=[str(i) for i in range(px.shape[0])])
    corr   = df_ret.corr()

    mask = np.triu(np.ones_like(corr, dtype=bool))
    plt.figure(figsize=(9,9))
    sns.heatmap(
        corr, mask=mask,
        cmap="coolwarm", center=0,
        vmin=-1, vmax=1,
        square=True,
        cbar_kws={"shrink":0.8}
    )
    plt.title("Instrument‐by‐Instrument Return Correlations (50×50)")
    plt.xlabel("Instrument")
    plt.ylabel("Instrument")
    plt.tight_layout()
    plt.show()

# ──────────────────── 7. PCA + Ward clustering ───────────────────────────── #
def pca_and_cluster(df: pd.DataFrame, blacklist: set[int]):
    X     = StandardScaler().fit_transform(df.values)
    comps = PCA(n_components=2, random_state=0).fit_transform(X)

    # Ward hierarchical clustering on the 2 PCs
    Z        = linkage(comps, method="ward")
    clusters = fcluster(Z, t=4, criterion="maxclust")

    # scatter: black ▲ for BLACK_LIST, colored ● for others
    plt.figure(figsize=(8,6))
    ax      = plt.gca()
    palette = sns.color_palette("tab10", max(clusters))

    for i, inst in enumerate(df.index):
        if inst in blacklist:
            ax.scatter(
                comps[i,0], comps[i,1],
                marker="^", s=100, c="black",
                label="BLACK_LIST" if "BLACK_LIST" not in ax.get_legend_handles_labels()[1] else ""
            )
        else:
            ax.scatter(
                comps[i,0], comps[i,1],
                marker="o", s=80, c=[palette[clusters[i]-1]],
                label="Others" if "Others" not in ax.get_legend_handles_labels()[1] else ""
            )

    ax.set_xlabel("PC‑1")
    ax.set_ylabel("PC‑2")
    ax.set_title("PCA Scatter with Ward Clusters\n(▲ = BLACK_LIST, ● = Others)")
    ax.legend()
    plt.tight_layout()
    plt.show()

    # dendrogram: leaves labeled 0–49, rotated for readability
    plt.figure(figsize=(12,5))
    dendrogram(
        Z,
        labels=[str(i) for i in df.index],
        leaf_rotation=90,
        leaf_font_size=10,
        color_threshold=0  # color every merge
    )
    plt.title("Hierarchical Clustering Dendrogram (Ward on PCs)")
    plt.ylabel("Linkage Distance")
    plt.tight_layout()
    plt.show()

# ──────────────────── 8. main entrypoint ─────────────────────────────────── #
def main():
    px      = load_prices()                   # (50, 1000)
    df_feat = build_feature_df(px)            # (50, 9)

    # print feature matrix
    print("\n=== Per‑instrument feature matrix ===")
    print(df_feat.round(4).to_string())

    # BLACK_LIST vs. others
    summary, pvals = compare_groups(df_feat, BLACK_LIST)
    print("\n=== BLACK_LIST vs. Others: Means & Differences ===")
    print(summary.round(4).to_string())
    print("\n=== Welch’s t‑test p‑values ===")
    print(pvals.round(4).to_string())

    # correlation heatmap
    correlation_heatmap(px)

    # PCA + scatter + dendrogram
    pca_and_cluster(df_feat, BLACK_LIST)

if __name__ == "__main__":
    main()


In [ ]:
#!/usr/bin/env python3
"""
dynamic_pip_variants.py

Defines a baseline PIP trader and five variants that “go hard” on the
identified BLACK_LIST instruments, then backtests all six models on
50 instruments × 1 000 timesteps and plots + compares their equity curves.
"""
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from typing import List, Optional, Tuple

# try to import Trader base class; fallback if missing
try:
    from Model.standard_template import Trader, export
except ImportError:
    class Trader:
        def __init__(self): pass
    def export(fn): return fn

# ───────────────── common parameters ──────────────────────────────── #
BLACK_LIST = {48, 45, 35, 22, 5, 23, 49}
POSITION_LIMIT   = 10_000
THR_MIN, THR_MAX, THR_STEP = 0.0, 0.700, 0.0005
THR_GRID = np.arange(THR_MIN, THR_MAX + THR_STEP, THR_STEP)
BEST_N = [
    200,200,400,200,400,400,100,300,100,200,
    100,500,100,300,200,100,200,100,200,300,
    100,200,500,100,400,100,100,400,100,100,
    400,500,300,200,400,500,100,400,100,100,
    100,300,400,500,100,400,500,100,500,300
]
BEST_W = [
     10,10,100,10,100,10,100,25,10,100,
    100,50,25,10,50,25,10,25,50,100,
    100,10,10,50,25,25,10,50,25,100,
     25,10,100,50,25,10,50,100,50,25,
     25,10,100,100,50,50,10,25,25,25
]

# ───────────────── helper functions ──────────────────────────────── #
def causal_update(px: float,
                  last_extreme: float,
                  direction: Optional[int],
                  thr: float) -> Tuple[int, float]:
    if direction in (0, None):
        move = (px - last_extreme) / last_extreme
        if abs(move) >= thr:
            return (1 if move > 0 else -1), px
        return 0, last_extreme
    if direction == 1:
        if px > last_extreme:
            return 1, px
        if (last_extreme - px) / last_extreme >= thr:
            return -1, px
        return 1, last_extreme
    # direction == -1
    if px < last_extreme:
        return -1, px
    if (px - last_extreme) / last_extreme >= thr:
        return 1, px
    return -1, last_extreme

def best_threshold(window: np.ndarray) -> float:
    best_thr, best_pnl = THR_GRID[0], -np.inf
    for thr in THR_GRID:
        dir_vec = np.zeros(len(window), np.int8)
        le, d = window[0], 0
        for k in range(1, len(window)):
            d, le = causal_update(window[k], le, d, thr)
            dir_vec[k] = d
        pnl = np.sum(dir_vec[:-1].astype(np.int32) *
                     POSITION_LIMIT * np.diff(window))
        if pnl > best_pnl:
            best_pnl, best_thr = pnl, thr
    return best_thr

# ───────────────── Base trader ──────────────────────────────────── #
class BaselineIgnore(Trader):
    """Ignore BLACK_LIST (stay flat)."""
    def __init__(self):
        super().__init__()
        # state = (n, w, next_cal, thr, dir, last_extreme)
        self._states = [(n, w, n, None, 0, None)
                        for n,w in zip(BEST_N, BEST_W)]

    @export
    def Alg(self, prcSoFar: np.ndarray) -> np.ndarray:
        n_inst, t = prcSoFar.shape
        pos = np.zeros(n_inst, np.int32)

        for i in range(n_inst):
            if i in BLACK_LIST:
                continue

            n, w, next_cal, thr, d, le = self._states[i]
            px = prcSoFar[i, -1]
            if le is None:
                le = px

            if t >= next_cal:
                window = prcSoFar[i, t-n:t]
                thr = best_threshold(window)
                next_cal += w
                print(f"[t={t:4d}] inst {i:2d} thr={thr:.4f}")

            if thr is None:
                self._states[i] = (n,w,next_cal,thr,d,le)
                continue

            d, le = causal_update(px, le, d, thr)
            pos[i] = d * POSITION_LIMIT
            self._states[i] = (n,w,next_cal,thr,d,le)

        return pos

# ───────────────── Variant 1: AggressiveSizing ────────────────────── #
class AggressiveSizing(BaselineIgnore):
    """2× position on BLACK_LIST when signaled."""
    @export
    def Alg(self, prcSoFar: np.ndarray) -> np.ndarray:
        pos = super().Alg(prcSoFar)
        for i in BLACK_LIST:
            if pos[i] != 0:
                pos[i] = 2 * POSITION_LIMIT * np.sign(pos[i])
        return pos

# ───────────────── Variant 2: AdaptiveThreshold ──────────────────── #
class AdaptiveThreshold(BaselineIgnore):
    """
    Use a tighter threshold grid (0→0.5) for BLACK_LIST only,
    by temporarily swapping THR_GRID in globals().
    """
    @export
    def Alg(self, prcSoFar: np.ndarray) -> np.ndarray:
        g = globals()
        orig_grid = g["THR_GRID"]
        # only shrink grid for identifed instruments
        alt_grid = np.arange(0.0, 0.5 + THR_STEP, THR_STEP)
        g["THR_GRID"] = alt_grid
        out = super().Alg(prcSoFar)
        g["THR_GRID"] = orig_grid
        return out

# ───────────────── Variant 3: VolatilityScaled ───────────────────── #
class VolatilityScaled(BaselineIgnore):
    """Scale non‑BL positions by 1/ATR14; BL stays full size."""
    @export
    def Alg(self, prcSoFar: np.ndarray) -> np.ndarray:
        pos = super().Alg(prcSoFar)
        for i in range(prcSoFar.shape[0]):
            if i in BLACK_LIST:
                continue
            window = prcSoFar[i, -15:]
            tr = np.abs(np.diff(window))
            atr14 = tr.mean() if len(tr) else 1.0
            scale = np.clip(1.0/atr14, 0.1, 5.0)
            pos[i] = int(scale * pos[i])
        return pos

# ───────────────── Variant 4: MomentumFiltered ───────────────────── #
class MomentumFiltered(BaselineIgnore):
    """BL trades only if 10‑bar SMA momentum agrees with signal."""
    @export
    def Alg(self, prcSoFar: np.ndarray) -> np.ndarray:
        pos = super().Alg(prcSoFar)
        for i in BLACK_LIST:
            p = prcSoFar[i]
            if len(p) < 11 or pos[i] == 0:
                pos[i] = 0
                continue
            sma_old = p[-11:-1].mean()
            sma_new = p[-10:].mean()
            if np.sign(sma_new - sma_old) != np.sign(pos[i]):
                pos[i] = 0
        return pos

# ───────────────── Variant 5: DrawdownStop ────────────────────────── #
class DrawdownStop(BaselineIgnore):
    """
    If BL PnL dips below −5% of POSITION_LIMIT, freeze until recovery.
    """
    def __init__(self):
        super().__init__()
        self._pnl = np.zeros(len(BEST_N))
        self._frozen = set()

    @export
    def Alg(self, prcSoFar: np.ndarray) -> np.ndarray:
        pos = super().Alg(prcSoFar)
        # update PnL
        if prcSoFar.shape[1] >= 2:
            delta = prcSoFar[:, -1] - prcSoFar[:, -2]
            self._pnl += pos * delta
        for i in BLACK_LIST:
            if i in self._frozen:
                if self._pnl[i] >= 0:
                    self._frozen.remove(i)
                else:
                    pos[i] = 0
            elif self._pnl[i] < -0.05 * POSITION_LIMIT:
                self._frozen.add(i)
                pos[i] = 0
        return pos

# ───────────────── backtester & runner ────────────────────────────── #
def load_prices(path="prices.txt") -> np.ndarray:
    raw = np.loadtxt(path)    # (1000×50)
    return raw.T              # (50×1000)

def backtest(model: Trader, px: np.ndarray) -> np.ndarray:
    _, T = px.shape
    cum_pnl = np.zeros(T)
    for t in range(1, T):
        pos = model.Alg(px[:, :t])
        cum_pnl[t] = cum_pnl[t-1] + (pos * (px[:, t] - px[:, t-1])).sum()
    return cum_pnl

if __name__ == "__main__":
    prices = load_prices()
    models = {
        "BaselineIgnore":    BaselineIgnore(),
        "AggressiveSizing":  AggressiveSizing(),
        "AdaptiveThreshold": AdaptiveThreshold(),
        "VolatilityScaled":  VolatilityScaled(),
        "MomentumFiltered":  MomentumFiltered(),
        "DrawdownStop":      DrawdownStop(),
    }

    results = {}
    for name, mdl in models.items():
        print(f"Running {name}…")
        results[name] = backtest(mdl, prices)

    df = pd.DataFrame(results)
    print("\nFinal PnLs:")
    print(df.iloc[-1].round(2).to_string())

    plt.figure(figsize=(10, 6))
    for col in df:
        plt.plot(df.index, df[col], label=col)
    plt.title("Equity Curves: Baseline vs. 5 Variants")
    plt.xlabel("Bar")
    plt.ylabel("Cumulative PnL")
    plt.legend()
    plt.tight_layout()
    plt.show()


In [ ]:
#!/usr/bin/env python3
"""
compare_baseline_vs_volatility_scaled.py

Loads 50×1000 price matrix from `prices.txt`, runs two PIP-based traders
– BaselineIgnore and VolatilityScaled – and computes per-instrument PnL
for each. Finally prints a DataFrame of final PnLs and summary comparisons.
"""
import numpy as np
import pandas as pd
from typing import Optional, Tuple

# Try to import Trader base class; fallback if missing
try:
    from Model.standard_template import Trader, export
except ImportError:
    class Trader:
        def __init__(self): pass
    def export(fn): return fn

# ─────────────── common parameters ────────────────────────────────────── #
BLACK_LIST = {48, 45, 35, 22, 5, 23, 49}
POSITION_LIMIT = 10_000

THR_MIN, THR_MAX, THR_STEP = 0.0, 0.700, 0.0005
THR_GRID = np.arange(THR_MIN, THR_MAX + THR_STEP, THR_STEP)

BEST_N = [
    200,200,400,200,400,400,100,300,100,200,
    100,500,100,300,200,100,200,100,200,300,
    100,200,500,100,400,100,100,400,100,100,
    400,500,300,200,400,500,100,400,100,100,
    100,300,400,500,100,400,500,100,500,300
]
BEST_W = [
     10,10,100,10,100,10,100,25,10,100,
    100,50,25,10,50,25,10,25,50,100,
    100,10,10,50,25,25,10,50,25,100,
     25,10,100,50,25,10,50,100,50,25,
     25,10,100,100,50,50,10,25,25,25
]

# ─────────────── helper functions ────────────────────────────────────── #
def causal_update(px: float,
                  last_extreme: float,
                  direction: Optional[int],
                  thr: float) -> Tuple[int, float]:
    if direction in (0, None):
        move = (px - last_extreme) / last_extreme
        if abs(move) >= thr:
            return (1 if move > 0 else -1), px
        return 0, last_extreme

    if direction == 1:
        if px > last_extreme:
            return 1, px
        if (last_extreme - px) / last_extreme >= thr:
            return -1, px
        return 1, last_extreme

    # direction == -1
    if px < last_extreme:
        return -1, px
    if (px - last_extreme) / last_extreme >= thr:
        return 1, px
    return -1, last_extreme

def best_threshold(window: np.ndarray) -> float:
    """
    Grid-search PIP threshold over THR_GRID for maximum in-sample PnL.
    """
    best_thr, best_pnl = THR_GRID[0], -np.inf
    for thr in THR_GRID:
        dir_vec = np.zeros(len(window), np.int8)
        le, d = window[0], 0
        for k in range(1, len(window)):
            d, le = causal_update(window[k], le, d, thr)
            dir_vec[k] = d
        pnl = np.sum(dir_vec[:-1].astype(np.int32) *
                     POSITION_LIMIT * np.diff(window))
        if pnl > best_pnl:
            best_pnl, best_thr = pnl, thr
    return best_thr

# ───────────────── Base trader ──────────────────────────────────────── #
class BaselineIgnore(Trader):
    """
    PIP trader that stays flat on BLACK_LIST instruments.
    """
    def __init__(self):
        super().__init__()
        # state for each instrument = (n, w, next_cal, thr, dir, last_extreme)
        self._states = [(n, w, n, None, 0, None)
                        for n, w in zip(BEST_N, BEST_W)]

    @export
    def Alg(self, prcSoFar: np.ndarray) -> np.ndarray:
        """
        prcSoFar: (50 × t_seen) array of prices up to current bar.
        Returns: (50,) positions.
        """
        n_inst, t_seen = prcSoFar.shape
        positions = np.zeros(n_inst, np.int32)

        for i in range(n_inst):
            if i in BLACK_LIST:
                continue

            n, w, next_cal, thr, d, le = self._states[i]
            px = prcSoFar[i, -1]
            if le is None:
                le = px

            # recalibrate threshold when due
            if t_seen >= next_cal:
                window = prcSoFar[i, t_seen-n : t_seen]
                thr = best_threshold(window)
                next_cal += w

            if thr is not None:
                d, le = causal_update(px, le, d, thr)
                positions[i] = d * POSITION_LIMIT

            self._states[i] = (n, w, next_cal, thr, d, le)

        return positions

# ─────────── Variant: VolatilityScaled ─────────────────────────────── #
class VolatilityScaled(BaselineIgnore):
    """
    Scales non-BL positions by 1/ATR14; BLACK_LIST remains flat.
    """
    @export
    def Alg(self, prcSoFar: np.ndarray) -> np.ndarray:
        pos = super().Alg(prcSoFar)
        # for each non-black instrument, adjust sizing by inverse ATR14
        for i in range(prcSoFar.shape[0]):
            if i in BLACK_LIST:
                continue
            window = prcSoFar[i, -15:]
            tr = np.abs(np.diff(window))
            atr14 = tr.mean() if len(tr) else 1.0
            scale = np.clip(1.0 / atr14, 0.1, 5.0)
            pos[i] = int(scale * pos[i])
        return pos

# ───────────────── backtester & runner ──────────────────────────────── #
def load_prices(path="prices.txt") -> np.ndarray:
    """
    Loads a 1000×50 file and returns a 50×1000 array: rows=instrument, cols=time.
    """
    raw = np.loadtxt(path)
    if raw.ndim != 2 or raw.shape[1] != 50:
        raise ValueError(f"Expected 1000×50 matrix; got {raw.shape}")
    return raw.T

def backtest_models(models: dict[str, Trader],
                    px: np.ndarray) -> dict[str, np.ndarray]:
    """
    Run each model over all bars; returns dict of final per-inst PnL arrays.
    """
    n_inst, T = px.shape
    # cumulative PnL per instrument, per model
    pnl = {name: np.zeros((n_inst, T), dtype=float)
           for name in models}

    # step through time
    for t in range(1, T):
        # compute positions at bar t using prices up to t
        positions = {name: mdl.Alg(px[:, :t]) for name, mdl in models.items()}
        # price change from t-1→t
        delta = px[:, t] - px[:, t-1]
        for name, pos in positions.items():
            pnl[name][:, t] = pnl[name][:, t-1] + pos * delta

    # extract final PnL per instrument
    final = {name: pnl[name][:, -1] for name in pnl}
    return final

if __name__ == "__main__":
    # load price data
    px = load_prices()

    # instantiate models
    models = {
        "BaselineIgnore": BaselineIgnore(),
        "VolatilityScaled": VolatilityScaled()
    }

    # run backtests
    final_pnls = backtest_models(models, px)

    # assemble results into DataFrame
    df = pd.DataFrame(final_pnls)
    df["Difference"] = df["VolatilityScaled"] - df["BaselineIgnore"]
    df["PctChange"] = df["Difference"] / df["BaselineIgnore"].replace(0, np.nan) * 100

    # print per-instrument PnL
    print("\nPer-Instrument Final PnL:")
    print(df.round(2).to_string())

    # summary statistics
    print("\nSummary Comparison:")
    summary = pd.Series({
        "Baseline total PnL":     df["BaselineIgnore"].sum(),
        "VolScaled total PnL":    df["VolatilityScaled"].sum(),
        "Total difference":       df["Difference"].sum(),
        "Avg per-inst ΔPnL":      df["Difference"].mean(),
        "Instruments improved":   (df["Difference"] > 0).sum(),
        "Instruments worsened":   (df["Difference"] < 0).sum(),
    })
    print(summary.round(2).to_string())


In [ ]:
#!/usr/bin/env python3
"""
pip_trader_variants.py

Defines five PIP‑based Trader variants using improved position‑sizing logic,
then backtests each from bar 1→1000 under realistic conditions (0.0005
commission per dollar traded).  Prints performance summaries and plots
equity curves.

Variants
--------
1. VolTargetTrader       – scale to target volatility (2% annual)
2. ClampedATRTrader      – ATR scaling clamped between 0.5× and 1.5×
3. SharpeScaledTrader    – scale by rolling Sharpe (0→2×)
4. MomentumFilteredTrader– only trade when 10‑bar SMA momentum agrees
5. CompositeScaledTrader – combine vol target with positive Sharpe

Usage
-----
Ensure `prices.txt` (1000 rows × 50 cols) is in working dir.  Run as:
    python pip_trader_variants.py
"""
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from typing import Optional, Tuple

# Try to import base Trader and export decorator; stub if missing
try:
    from Model.standard_template import Trader, export
except ImportError:
    class Trader:
        def __init__(self): pass
    def export(fn): return fn

# ───────────────── common parameters ──────────────────────────────── #
BLACK_LIST      = {48, 45, 35, 22, 5, 23, 49}
POSITION_LIMIT  = 10_000
COMMISSION_RATE = 0.0005

THR_MIN, THR_MAX, THR_STEP = 0.0, 0.700, 0.0005
THR_GRID = np.arange(THR_MIN, THR_MAX + THR_STEP, THR_STEP)

BEST_N = [
    200,200,400,200,400,400,100,300,100,200,
    100,500,100,300,200,100,200,100,200,300,
    100,200,500,100,400,100,100,400,100,100,
    400,500,300,200,400,500,100,400,100,100,
    100,300,400,500,100,400,500,100,500,300
]
BEST_W = [
     10,10,100,10,100,10,100,25,10,100,
    100,50,25,10,50,25,10,25,50,100,
    100,10,10,50,25,25,10,50,25,100,
     25,10,100,50,25,10,50,100,50,25,
     25,10,100,100,50,50,10,25,25,25
]

# ──────────────────── helper functions ────────────────────────────── #
def causal_update(px: float,
                  last_extreme: float,
                  direction: Optional[int],
                  thr: float) -> Tuple[int, float]:
    if direction in (0, None):
        move = (px - last_extreme) / last_extreme
        if abs(move) >= thr:
            return (1 if move > 0 else -1), px
        return 0, last_extreme
    if direction == 1:
        if px > last_extreme:
            return 1, px
        if (last_extreme - px) / last_extreme >= thr:
            return -1, px
        return 1, last_extreme
    if px < last_extreme:
        return -1, px
    if (px - last_extreme) / last_extreme >= thr:
        return 1, px
    return -1, last_extreme

def best_threshold(window: np.ndarray) -> float:
    best_thr, best_pnl = THR_GRID[0], -np.inf
    for thr in THR_GRID:
        dir_vec = np.zeros(len(window), np.int8)
        le, d = window[0], 0
        for k in range(1, len(window)):
            d, le = causal_update(window[k], le, d, thr)
            dir_vec[k] = d
        pnl = np.sum(dir_vec[:-1].astype(np.int32) *
                     POSITION_LIMIT * np.diff(window))
        if pnl > best_pnl:
            best_pnl, best_thr = pnl, thr
    return best_thr

def realised_vol(returns: np.ndarray) -> float:
    return returns.std(ddof=0) * np.sqrt(252)

def sharpe_ratio(returns: np.ndarray) -> float:
    sigma = returns.std(ddof=0)
    return 0.0 if sigma == 0 else (returns.mean() / sigma) * np.sqrt(252)

# ────────────────── Base PIP Trader ───────────────────────────────── #
class BasePIPTrader(Trader):
    class _State:
        __slots__ = ("n","w","next_cal","thr","dir","le")
        def __init__(self, n:int, w:int):
            self.n        = n
            self.w        = w
            self.next_cal = n
            self.thr      = None
            self.dir      = 0
            self.le       = None

    def __init__(self):
        super().__init__()
        self._states = [self._State(n,w) for n,w in zip(BEST_N, BEST_W)]
        print(f"{self.__class__.__name__} initialized.")

    @export
    def Alg(self, prcSoFar: np.ndarray) -> np.ndarray:
        n_inst, t_seen = prcSoFar.shape
        pos = np.zeros(n_inst, np.int32)

        for i in range(n_inst):
            if i in BLACK_LIST:
                continue

            st = self._states[i]
            px = prcSoFar[i, -1]
            if st.le is None:
                st.le = px

            if t_seen >= st.next_cal:
                window = prcSoFar[i, t_seen-st.n : t_seen]
                st.thr = best_threshold(window)
                st.next_cal += st.w
                print(f"[t={t_seen:4d}] {self.__class__.__name__} inst {i:2d} thr={st.thr:.4f}")

            if st.thr is None:
                continue

            st.dir, st.le = causal_update(px, st.le, st.dir, st.thr)
            raw_pos = st.dir * POSITION_LIMIT

            scale = self.get_scale(i, prcSoFar, st)
            pos[i] = int(raw_pos * scale)

        return pos

    def get_scale(self, i:int, prcSoFar:np.ndarray, st:_State) -> float:
        return 1.0

# ────────────────── Variant 1: VolTargetTrader ────────────────────── #
class VolTargetTrader(BasePIPTrader):
    def __init__(self, target_vol: float = 0.02):
        super().__init__()
        self.target_vol = target_vol

    def get_scale(self, i:int, prcSoFar:np.ndarray, st:BasePIPTrader._State) -> float:
        window = prcSoFar[i, -st.n:]
        ret = np.diff(window) / window[:-1]
        if len(ret) > 1:
            vol = realised_vol(ret)
        else:
            vol = 1.0
        scale = np.clip(self.target_vol / vol, 0.1, 5.0)
        return scale

# ────────────────── Variant 2: ClampedATRTrader ───────────────────── #
class ClampedATRTrader(BasePIPTrader):
    def get_scale(self, i:int, prcSoFar:np.ndarray, st:BasePIPTrader._State) -> float:
        window = prcSoFar[i, -15:]
        tr = np.abs(np.diff(window))
        atr14 = tr.mean() if len(tr)>0 else 1.0
        return np.clip(1.0 / atr14, 0.5, 1.5)

# ────────────────── Variant 3: SharpeScaledTrader ─────────────────── #
class SharpeScaledTrader(BasePIPTrader):
    def get_scale(self, i:int, prcSoFar:np.ndarray, st:BasePIPTrader._State) -> float:
        window = prcSoFar[i, -st.n:]
        ret = np.diff(window) / window[:-1]
        sr = sharpe_ratio(ret) if len(ret)>1 else 0.0
        return np.clip(sr, 0.0, 2.0)

# ──────────────── Variant 4: MomentumFilteredTrader ────────────────── #
class MomentumFilteredTrader(BasePIPTrader):
    def get_scale(self, i:int, prcSoFar:np.ndarray, st:BasePIPTrader._State) -> float:
        p = prcSoFar[i]
        if len(p) < 11 or st.dir == 0:
            return 1.0
        sma_old = p[-11:-1].mean()
        sma_new = p[-10:].mean()
        return 1.0 if np.sign(sma_new - sma_old) == np.sign(st.dir) else 0.0

# ───────────── Variant 5: CompositeScaledTrader ────────────────────── #
class CompositeScaledTrader(BasePIPTrader):
    def __init__(self, target_vol: float = 0.02):
        super().__init__()
        self.target_vol = target_vol

    def get_scale(self, i:int, prcSoFar:np.ndarray, st:BasePIPTrader._State) -> float:
        window = prcSoFar[i, -st.n:]
        ret    = np.diff(window) / window[:-1]
        if len(ret) > 1:
            vol = realised_vol(ret)
            sr  = sharpe_ratio(ret)
        else:
            vol, sr = 1.0, 0.0
        scale = (self.target_vol / vol) * max(sr, 0.0)
        return np.clip(scale, 0.1, 5.0)

# ───────────────────── backtester & runner ─────────────────────────── #
def load_prices(path="prices.txt") -> np.ndarray:
    raw = np.loadtxt(path)  # (1000,50)
    if raw.ndim!=2 or raw.shape[1]!=50:
        raise ValueError(f"Expected 1000×50 matrix; got {raw.shape}")
    return raw.T           # (50,1000)

def backtest(trader: Trader, px: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
    n_inst, T = px.shape
    cum_pnl   = np.zeros(T, dtype=float)
    daily_pnl = np.zeros(T, dtype=float)
    prev_pos  = np.zeros(n_inst, dtype=float)

    for t in range(1, T):
        pos       = trader.Alg(px[:, :t])
        delta     = px[:, t] - px[:, t-1]
        gross_pnl = np.sum(prev_pos * delta)
        trades    = pos - prev_pos
        trade_val = np.sum(np.abs(trades) * px[:, t])
        comm      = COMMISSION_RATE * trade_val
        pnl       = gross_pnl - comm

        cum_pnl[t]   = cum_pnl[t-1] + pnl
        daily_pnl[t] = pnl
        prev_pos     = pos

    return cum_pnl, daily_pnl

def performance_summary(cum: np.ndarray, daily: np.ndarray) -> dict:
    mean     = daily.mean()
    std      = daily.std(ddof=1)
    sharpe   = (mean/std) * np.sqrt(252) if std>0 else 0.0
    win_rate = np.mean(daily>0) * 100
    return {
        "Final PnL":           cum[-1],
        "Mean daily PnL":      mean,
        "Std daily PnL":       std,
        "Ann. Sharpe":         sharpe,
        "Win Rate (%)":        win_rate,
    }

if __name__ == "__main__":
    prices = load_prices()
    print(f"Loaded {prices.shape[0]} instruments × {prices.shape[1]} bars\n")

    models = {
        "VolTarget":       VolTargetTrader(),
        "ClampedATR":      ClampedATRTrader(),
        "SharpeScaled":    SharpeScaledTrader(),
        "MomentumFilter":  MomentumFilteredTrader(),
        "Composite":       CompositeScaledTrader(),
    }

    results, summaries = {}, {}
    for name, mdl in models.items():
        print(f"=== Running {name} ===")
        cum, daily = backtest(mdl, prices)
        results[name]   = cum
        summaries[name] = performance_summary(cum, daily)
        print(f"  Final PnL: {cum[-1]:.2f}, Ann Sharpe: {summaries[name]['Ann. Sharpe']:.2f}\n")

    df_sum = pd.DataFrame(summaries).T.round(2)
    print("Performance Summary:\n", df_sum.to_string(), "\n")

    plt.figure(figsize=(10,6))
    for name, cum in results.items():
        plt.plot(cum, label=name)
    plt.title("Equity Curves: Five PIP Variants")
    plt.xlabel("Bar")
    plt.ylabel("Cumulative PnL")
    plt.legend()
    plt.tight_layout()
    plt.show()


In [ ]:
#!/usr/bin/env python3
"""
advanced_pip_variants.py

Implements five new causal, cross‑instrument PIP‐style traders designed
to boost performance (~3× baseline) while avoiding overfitting. Each
uses a distinct heuristic and trades all 50 instruments.

Variants:
1. CrossSectionMomentumTrader  – long top 5 momentum, short bottom 5
2. BreakoutTrader               – 20‑day high/low breakout
3. MeanReversionTrader          – 20‑day z‑score mean reversion
4. RiskParityMomentumTrader     – cross‑sectional momentum weighted by inverse vol
5. EnsembleTrader               – average positions of the above four

Backtests from bar 1→999 on 50 × 1000 price matrix with 0.0005 commission.
Prints per‑model summaries and plots equity curves.
"""
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from typing import Tuple

# Trader base stub
try:
    from Model.standard_template import Trader, export
except ImportError:
    class Trader:
        def __init__(self): pass
    def export(fn): return fn

# ───────────────── common settings ──────────────────────────────── #
BLACK_LIST      = {48, 45, 35, 22, 5, 23, 49}
POSITION_LIMIT  = 10_000
COMMISSION_RATE = 0.0005

LOOKBACK_MOM     = 20
LOOKBACK_Z       = 20
LOOKBACK_BREAK   = 20
RISK_PERIOD_VOL  = 20
N_LONG_SHORT     = 5  # for cross‐momentum

# ────────────────── Helper functions ───────────────────────────── #
def realized_vol(returns: np.ndarray) -> float:
    return returns.std(ddof=0) * np.sqrt(252)

def zscore(x: np.ndarray) -> np.ndarray:
    return (x - x.mean()) / x.std(ddof=0)

# ──────────────── 1. Cross‑Section Momentum ─────────────────────── #
class CrossSectionMomentumTrader(Trader):
    """Long top N_LONG_SHORT by 20‑day return, short bottom N."""
    @export
    def Alg(self, prcSoFar: np.ndarray) -> np.ndarray:
        t = prcSoFar.shape[1]
        pos = np.zeros(prcSoFar.shape[0], dtype=int)

        if t < LOOKBACK_MOM+1:
            return pos

        # compute 20‑day simple returns
        past = prcSoFar[:, t-LOOKBACK_MOM-1:t-1]
        ret20 = (past[:, -1] - past[:, 0]) / past[:, 0]
        # rank
        idx = np.argsort(ret20)
        longs  = idx[-N_LONG_SHORT:]
        shorts = idx[:N_LONG_SHORT]

        for i in longs:
            if i not in BLACK_LIST:
                pos[i] =  POSITION_LIMIT
        for i in shorts:
            if i not in BLACK_LIST:
                pos[i] = -POSITION_LIMIT

        return pos

# ────────────────── 2. Breakout Trader ──────────────────────────── #
class BreakoutTrader(Trader):
    """20‑day high/low breakout: long above high, short below low."""
    @export
    def Alg(self, prcSoFar: np.ndarray) -> np.ndarray:
        t = prcSoFar.shape[1]
        pos = np.zeros(prcSoFar.shape[0], dtype=int)
        if t < LOOKBACK_BREAK+1:
            return pos

        window = prcSoFar[:, t-LOOKBACK_BREAK-1:t-1]
        highs, lows = window.max(axis=1), window.min(axis=1)
        cur = prcSoFar[:, -1]

        for i in range(prcSoFar.shape[0]):
            if i in BLACK_LIST:
                continue
            if cur[i] > highs[i]:
                pos[i] = POSITION_LIMIT
            elif cur[i] < lows[i]:
                pos[i] = -POSITION_LIMIT
        return pos

# ──────────────── 3. Mean Reversion Trader ─────────────────────── #
class MeanReversionTrader(Trader):
    """20‑day z‑score: short >+1.5, long <-1.5."""
    @export
    def Alg(self, prcSoFar: np.ndarray) -> np.ndarray:
        t = prcSoFar.shape[1]
        pos = np.zeros(prcSoFar.shape[0], dtype=int)
        if t < LOOKBACK_Z+1:
            return pos

        window = prcSoFar[:, t-LOOKBACK_Z-1:t-1]
        zs = zscore(window)[:, -1]

        for i in range(prcSoFar.shape[0]):
            if i in BLACK_LIST:
                continue
            if zs[i] > 1.5:
                pos[i] = -POSITION_LIMIT
            elif zs[i] < -1.5:
                pos[i] =  POSITION_LIMIT
        return pos

# ──────────── 4. Risk‑Parity Momentum Trader ────────────────────── #
class RiskParityMomentumTrader(Trader):
    """
    Cross‐section momentum weighted by inv(vol):
    weight_i ∝ ret20_i / vol_i, normalize sum(|w|)=1.
    """
    @export
    def Alg(self, prcSoFar: np.ndarray) -> np.ndarray:
        t = prcSoFar.shape[1]
        pos = np.zeros(prcSoFar.shape[0], dtype=float)
        if t < max(LOOKBACK_MOM, RISK_PERIOD_VOL)+1:
            return pos.astype(int)

        # 20‑day returns
        past20 = prcSoFar[:, t-LOOKBACK_MOM-1:t-1]
        ret20  = (past20[:, -1] - past20[:, 0]) / past20[:, 0]
        # vol
        pastV = prcSoFar[:, t-RISK_PERIOD_VOL-1:t-1]
        retV  = np.diff(pastV) / pastV[:, :-1]
        vol    = np.where(retV.shape[1]>1,
                          np.std(retV, axis=1, ddof=0)*np.sqrt(252),
                          1.0)
        # raw weight
        w = ret20 / np.where(vol>0, vol, 1.0)
        # zero out black list
        w[list(BLACK_LIST)] = 0.0
        # normalize so sum(abs)=1
        if np.sum(np.abs(w)) > 0:
            w = w / np.sum(np.abs(w))
        # allocate positions
        pos = w * POSITION_LIMIT * len(w)  # total capital scaled
        return pos.astype(int)

# ────────────────── 5. Ensemble Trader ──────────────────────────── #
class EnsembleTrader(Trader):
    """
    Average the four above signals, then threshold to ±POSITION_LIMIT.
    """
    def __init__(self):
        super().__init__()
        self._sub = [
            CrossSectionMomentumTrader(),
            BreakoutTrader(),
            MeanReversionTrader(),
            RiskParityMomentumTrader(),
        ]

    @export
    def Alg(self, prcSoFar: np.ndarray) -> np.ndarray:
        N = prcSoFar.shape[0]
        agg = np.zeros(N, dtype=float)
        for mdl in self._sub:
            agg += mdl.Alg(prcSoFar)
        agg /= len(self._sub)
        # threshold:
        pos = np.zeros(N, dtype=int)
        for i in range(N):
            if i in BLACK_LIST:
                continue
            if agg[i] > 0:
                pos[i] = POSITION_LIMIT
            elif agg[i] < 0:
                pos[i] = -POSITION_LIMIT
        return pos

# ────────────────── Backtester & Runner ─────────────────────────── #
def load_prices(path="prices.txt") -> np.ndarray:
    raw = np.loadtxt(path)      # (1000×50)
    if raw.ndim!=2 or raw.shape[1]!=50:
        raise ValueError(f"Expected 1000×50, got {raw.shape}")
    return raw.T                # (50×1000)

def backtest(trader: Trader, px: np.ndarray) -> Tuple[np.ndarray,np.ndarray]:
    n, T = px.shape
    cum = np.zeros(T)
    prev = np.zeros(n)
    daily = np.zeros(T)
    for t in range(1,T):
        pos = trader.Alg(px[:,:t])
        delta = px[:,t] - px[:,t-1]
        pnl_gross = np.dot(prev, delta)
        trades = pos - prev
        trade_val = np.dot(np.abs(trades), px[:,t])
        comm = COMMISSION_RATE * trade_val
        pnl = pnl_gross - comm
        cum[t]  = cum[t-1] + pnl
        daily[t]= pnl
        prev = pos
    return cum, daily

def perf_summary(cum: np.ndarray, daily: np.ndarray, name: str):
    mean = daily.mean(); std = daily.std(ddof=1)
    sr   = (mean/std)*np.sqrt(252) if std>0 else 0.0
    wr   = np.mean(daily>0)*100
    print(f"{name:30s} | Final PnL: {cum[-1]:9.0f} | Sharpe: {sr:5.2f} | Win%: {wr:5.1f}")

if __name__=="__main__":
    prices = load_prices()
    print(f"Loaded prices: {prices.shape[0]} instruments × {prices.shape[1]} bars\n")

    models = {
        "CrossSectionMomentum": CrossSectionMomentumTrader(),
        "Breakout":             BreakoutTrader(),
        "MeanReversion":        MeanReversionTrader(),
        "RiskParityMomentum":   RiskParityMomentumTrader(),
        "Ensemble":             EnsembleTrader(),
    }

    results, dailies = {}, {}
    for name, mdl in models.items():
        print(f"Running {name}...", end=" ")
        cum, daily = backtest(mdl, prices)
        results[name], dailies[name] = cum, daily
        print("Done")

    print("\nPerformance Summary:")
    for name in models:
        perf_summary(results[name], dailies[name], name)

    # plot equity
    plt.figure(figsize=(10,6))
    for name,cum in results.items():
        plt.plot(cum, label=name)
    plt.title("Equity Curves: New PIP Variants")
    plt.xlabel("Bar")
    plt.ylabel("Cumulative PnL")
    plt.legend()
    plt.tight_layout()
    plt.show()


In [ ]:
#!/usr/bin/env python3
"""
advanced_ensemble_pip_traders.py

Defines and backtests ten causal, cross‑instrument PIP‑style traders:
 1. BasePIPTrader
 2. ATRClampTrader
 3. VolTargetTrader
 4. SharpeScaledTrader
 5. MomentumFilteredTrader
 6. CompositeScaledTrader
 7. BetaScaledTrader
 8. SkewScaledTrader
 9. DrawdownStopTrader
10. MasterEnsembleTrader

Each uses a distinct scaling or filtering heuristic, trades all 50
instruments except a static BLACK_LIST, and is evaluated from bar 1→999
on 50×1000 price data with $0.0005 commission.  Prints a performance
summary and plots equity curves.
"""
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from typing import Optional, Tuple

# ─────────────────── Trader base stub ─────────────────────────────── #
try:
    from Model.standard_template import Trader, export
except ImportError:
    class Trader:
        def __init__(self): pass
    def export(fn): return fn

# ─────────────────── common settings ──────────────────────────────── #
BLACK_LIST      = {48, 45, 35, 22, 5, 23, 49}
POSITION_LIMIT  = 10_000
COMMISSION_RATE = 0.0005

# PIP threshold grid
THR_GRID = np.arange(0.0, 0.700 + 0.0005, 0.0005)
# per‑inst lookback & cadence
BEST_N = [200,200,400,200,400,400,100,300,100,200,
          100,500,100,300,200,100,200,100,200,300,
          100,200,500,100,400,100,100,400,100,100,
          400,500,300,200,400,500,100,400,100,100,
          100,300,400,500,100,400,500,100,500,300]
BEST_W = [10,10,100,10,100,10,100,25,10,100,
          100,50,25,10,50,25,10,25,50,100,
          100,10,10,50,25,25,10,50,25,100,
          25,10,100,50,25,10,50,100,50,25,
          25,10,100,100,50,50,10,25,25,25]

# ─────────────────── helper functions ─────────────────────────────── #
def causal_update(px: float,
                  last_extreme: float,
                  direction: Optional[int],
                  thr: float) -> Tuple[int, float]:
    if direction in (0, None):
        move = (px - last_extreme) / last_extreme
        if abs(move) >= thr:
            return (1 if move > 0 else -1), px
        return 0, last_extreme
    if direction == 1:
        if px > last_extreme:
            return 1, px
        if (last_extreme - px)/last_extreme >= thr:
            return -1, px
        return 1, last_extreme
    # direction == -1
    if px < last_extreme:
        return -1, px
    if (px - last_extreme)/last_extreme >= thr:
        return 1, px
    return -1, last_extreme

def best_threshold(window: np.ndarray) -> float:
    best_thr, best_pnl = THR_GRID[0], -np.inf
    for thr in THR_GRID:
        dir_vec = np.zeros(len(window), np.int8)
        le, d = window[0], 0
        for k in range(1, len(window)):
            d, le = causal_update(window[k], le, d, thr)
            dir_vec[k] = d
        pnl = (dir_vec[:-1].astype(np.int32) *
               POSITION_LIMIT * np.diff(window)).sum()
        if pnl > best_pnl:
            best_pnl, best_thr = pnl, thr
    return best_thr

def realized_vol(returns: np.ndarray) -> float:
    return returns.std(ddof=0) * np.sqrt(252)

def sharpe_ratio(returns: np.ndarray) -> float:
    sigma = returns.std(ddof=0)
    return 0.0 if sigma==0 else (returns.mean()/sigma)*np.sqrt(252)

def skewness(returns: np.ndarray) -> float:
    m = returns.mean()
    s = returns.std(ddof=0)
    return ((returns - m)**3).mean()/s**3 if s>0 else 0.0

# ──────────── 1. BasePIPTrader ────────────────────────────────────── #
class BasePIPTrader(Trader):
    class _State:
        __slots__ = ("n","w","next_cal","thr","dir","le")
        def __init__(self, n,w):
            self.n, self.w = n,w
            self.next_cal  = n
            self.thr       = None
            self.dir       = 0
            self.le        = None

    def __init__(self):
        super().__init__()
        self._states = [self._State(n,w) for n,w in zip(BEST_N,BEST_W)]
        print(f"{self.__class__.__name__} initialized.")

    @export
    def Alg(self, prc: np.ndarray) -> np.ndarray:
        N, t = prc.shape
        pos = np.zeros(N, np.int32)
        for i in range(N):
            if i in BLACK_LIST:
                continue
            st = self._states[i]
            px = prc[i,-1]
            if st.le is None:
                st.le = px
            if t >= st.next_cal:
                win = prc[i, t-st.n:t]
                st.thr = best_threshold(win)
                st.next_cal += st.w
                print(f"[t={t}] {self.__class__.__name__} inst {i} thr={st.thr:.4f}")
            if st.thr is None:
                continue
            st.dir, st.le = causal_update(px, st.le, st.dir, st.thr)
            raw = st.dir * POSITION_LIMIT
            scale = self.get_scale(i, prc, st)
            pos[i] = int(raw * scale)
        return pos

    def get_scale(self, i:int, prc:np.ndarray, st:_State) -> float:
        return 1.0

# ──────────── 2. ATRClampTrader ────────────────────────────────────── #
class ATRClampTrader(BasePIPTrader):
    """Scales by 1/ATR14 clamped [0.5,1.5]."""
    def get_scale(self, i, prc, st):
        window = prc[i, -15:]
        tr     = np.abs(np.diff(window))
        atr14  = tr.mean() if len(tr)>0 else 1.0
        return float(np.clip(1.0/atr14, 0.5, 1.5))

# ──────────── 3. VolTargetTrader ──────────────────────────────────── #
class VolTargetTrader(BasePIPTrader):
    """Scale to target vol 2%."""
    def __init__(self, target_vol=0.02):
        super().__init__()
        self.tv = target_vol
    def get_scale(self, i, prc, st):
        win = prc[i, -st.n:]
        ret = np.diff(win)/win[:-1]
        vol = realized_vol(ret) if len(ret)>1 else 1.0
        return float(np.clip(self.tv/vol, 0.1,5.0))

# ──────────── 4. SharpeScaledTrader ────────────────────────────────── #
class SharpeScaledTrader(BasePIPTrader):
    """Scale by rolling Sharpe [0,2]."""
    def get_scale(self, i, prc, st):
        win = prc[i, -st.n:]
        ret = np.diff(win)/win[:-1]
        sr  = sharpe_ratio(ret) if len(ret)>1 else 0.0
        return float(np.clip(sr, 0.0,2.0))

# ──────────── 5. MomentumFilteredTrader ────────────────────────────── #
class MomentumFilteredTrader(BasePIPTrader):
    """Trade only when 10‑bar SMA momentum aligns."""
    def get_scale(self, i, prc, st):
        p = prc[i]
        if len(p)<11 or st.dir==0:
            return 1.0
        sma_old = p[-11:-1].mean()
        sma_new = p[-10:].mean()
        return 1.0 if np.sign(sma_new-sma_old)==st.dir else 0.0

# ──────────── 6. CompositeScaledTrader ────────────────────────────── #
class CompositeScaledTrader(BasePIPTrader):
    """vol‑target × positive Sharpe, clamped."""
    def __init__(self, tv=0.02):
        super().__init__()
        self.tv = tv
    def get_scale(self, i, prc, st):
        win = prc[i, -st.n:]
        ret = np.diff(win)/win[:-1]
        vol = realized_vol(ret) if len(ret)>1 else 1.0
        sr  = sharpe_ratio(ret) if len(ret)>1 else 0.0
        sc = (self.tv/vol)*max(sr,0.0)
        return float(np.clip(sc, 0.1,5.0))

# ──────────── 7. BetaScaledTrader ─────────────────────────────────── #
class BetaScaledTrader(BasePIPTrader):
    """Scale by inverse beta to cross‑section market."""
    def get_scale(self, i, prc, st):
        n, t = prc.shape
        win = prc[i, -st.n:]
        ret_i = np.diff(win)/win[:-1]
        # market return = average of all
        market = prc[:, -st.n-1:-1]
        ret_m = (market[:,1:]-market[:,:-1])/market[:,:-1]
        mkt_ret = ret_m.mean(axis=0)
        cov = np.cov(ret_i, mkt_ret, ddof=0)[0,1]
        var = mkt_ret.var(ddof=0)
        beta = cov/var if var>0 else 1.0
        invb = 1.0/beta if beta!=0 else 1.0
        return float(np.clip(invb, 0.5,2.0))

# ──────────── 8. SkewScaledTrader ──────────────────────────────────── #
class SkewScaledTrader(BasePIPTrader):
    """Scale by (1+skewness), clamped."""
    def get_scale(self, i, prc, st):
        win = prc[i, -st.n:]
        ret = np.diff(win)/win[:-1]
        sk  = skewness(ret) if len(ret)>1 else 0.0
        return float(np.clip(1.0+sk, 0.5,2.0))

# ──────────── 9. DrawdownStopTrader ────────────────────────────────── #
class DrawdownStopTrader(BasePIPTrader):
    """Freeze instrument if price drawdown >10%."""
    def __init__(self):
        super().__init__()
        for st in self._states:
            st.peak = None
        self._frozen = set()

    @export
    def Alg(self, prc: np.ndarray) -> np.ndarray:
        pos = super().Alg(prc).astype(float)
        N, t = prc.shape
        for i in range(N):
            if i in BLACK_LIST:
                continue
            px = prc[i,-1]
            st = self._states[i]
            if st.peak is None:
                st.peak = px
            st.peak = max(st.peak, px)
            dd = (px - st.peak)/st.peak
            if i in self._frozen:
                if dd >= 0:
                    self._frozen.remove(i)
                else:
                    pos[i] = 0.0
            elif dd < -0.10:
                self._frozen.add(i)
                pos[i] = 0.0
        return pos.astype(int)

# ────────────10. MasterEnsembleTrader ──────────────────────────────── #
class MasterEnsembleTrader(Trader):
    """Average signals from the nine PIP variants and threshold."""
    def __init__(self):
        super().__init__()
        self.mods = [
            BasePIPTrader(),
            ATRClampTrader(),
            VolTargetTrader(),
            SharpeScaledTrader(),
            MomentumFilteredTrader(),
            CompositeScaledTrader(),
            BetaScaledTrader(),
            SkewScaledTrader(),
            DrawdownStopTrader(),
        ]
        print("MasterEnsembleTrader ready.")

    @export
    def Alg(self, prc: np.ndarray) -> np.ndarray:
        N = prc.shape[0]
        agg = np.zeros(N, dtype=float)
        for m in self.mods:
            agg += m.Alg(prc)
        agg /= len(self.mods)
        pos = np.zeros(N, dtype=int)
        for i in range(N):
            if i in BLACK_LIST:
                continue
            if agg[i] > 0:
                pos[i] = POSITION_LIMIT
            elif agg[i] < 0:
                pos[i] = -POSITION_LIMIT
        return pos

# ─────────────────── Backtest & summary ───────────────────────────── #
def load_prices(path="prices.txt") -> np.ndarray:
    raw = np.loadtxt(path)
    if raw.ndim!=2 or raw.shape[1]!=50:
        raise ValueError(f"Expected 1000×50 data, got {raw.shape}")
    return raw.T  # (50×1000)

def backtest(trader: Trader, px: np.ndarray) -> Tuple[np.ndarray,np.ndarray]:
    n, T = px.shape
    cum   = np.zeros(T)
    daily = np.zeros(T)
    prev  = np.zeros(n)
    for t in range(1, T):
        pos = trader.Alg(px[:,:t])
        dpx = px[:,t] - px[:,t-1]
        gross = prev.dot(dpx)
        trades= pos - prev
        tv    = np.abs(trades).dot(px[:,t])
        comm  = COMMISSION_RATE * tv
        pnl   = gross - comm
        cum[t]= cum[t-1] + pnl
        daily[t]= pnl
        prev   = pos
    return cum, daily

def perf_summary(name, cum, daily):
    mean, std = daily.mean(), daily.std(ddof=1)
    sharpe = (mean/std)*np.sqrt(252) if std>0 else 0.0
    wr = np.mean(daily>0)*100
    print(f"{name:25s} | Final PnL: {cum[-1]:9.0f} | Sharpe: {sharpe:5.2f} | Win%: {wr:5.1f}")

if __name__ == "__main__":
    prices = load_prices()
    print(f"Prices loaded: {prices.shape[0]} instruments × {prices.shape[1]} bars\n")

    traders = {
        "BasePIP":          BasePIPTrader(),
        "ATRClamp":         ATRClampTrader(),
        "VolTarget":        VolTargetTrader(),
        "SharpeScaled":     SharpeScaledTrader(),
        "MomentumFilter":   MomentumFilteredTrader(),
        "CompositeScaled":  CompositeScaledTrader(),
        "BetaScaled":       BetaScaledTrader(),
        "SkewScaled":       SkewScaledTrader(),
        "DrawdownStop":     DrawdownStopTrader(),
        "MasterEnsemble":   MasterEnsembleTrader(),
    }

    results, dailies = {}, {}
    for name, tr in traders.items():
        print(f"Running {name}…")
        cum, daily = backtest(tr, prices)
        results[name], dailies[name] = cum, daily

    print("\nPerformance Summary:")
    for name in traders:
        perf_summary(name, results[name], dailies[name])

    plt.figure(figsize=(12,6))
    for name, cum in results.items():
        plt.plot(cum, label=name)
    plt.title("Equity Curves: 10 PIP‑Style Traders")
    plt.xlabel("Bar")
    plt.ylabel("Cumulative PnL")
    plt.legend(ncol=2, fontsize="small")
    plt.tight_layout()
    plt.show()


In [ ]:
#!/usr/bin/env python3
"""
advanced_ensemble_pip_traders_fixed.py

Defines and backtests ten causal, cross‑instrument PIP‑style traders:
 1. BasePIPTrader
 2. ATRClampTrader
 3. VolTargetTrader
 4. SharpeScaledTrader
 5. MomentumFilteredTrader
 6. CompositeScaledTrader
 7. BetaScaledTrader
 8. SkewScaledTrader
 9. DrawdownStopTrader  ← fixed to track peaks externally
10. MasterEnsembleTrader

Backtests from bar 1→999 on 50×1000 price data with $0.0005 commission.
Prints performance summary and plots equity curves.
"""
import numpy as np
import matplotlib.pyplot as plt
from typing import Optional, Tuple

# ─────────────────── Trader base stub ─────────────────────────────── #
try:
    from Model.standard_template import Trader, export
except ImportError:
    class Trader:
        def __init__(self): pass
    def export(fn): return fn

# ─────────────────── common settings ──────────────────────────────── #
BLACK_LIST      = {48, 45, 35, 22, 5, 23, 49}
POSITION_LIMIT  = 10_000
COMMISSION_RATE = 0.0005

THR_GRID = np.arange(0.0, 0.700 + 0.0005, 0.0005)
BEST_N   = [200,200,400,200,400,400,100,300,100,200,
            100,500,100,300,200,100,200,100,200,300,
            100,200,500,100,400,100,100,400,100,100,
            400,500,300,200,400,500,100,400,100,100,
            100,300,400,500,100,400,500,100,500,300]
BEST_W   = [10,10,100,10,100,10,100,25,10,100,
            100,50,25,10,50,25,10,25,50,100,
            100,10,10,50,25,25,10,50,25,100,
            25,10,100,50,25,10,50,100,50,25,
            25,10,100,100,50,50,10,25,25,25]

# ─────────────────── helper functions ─────────────────────────────── #
def causal_update(px: float, last_extreme: float,
                  direction: Optional[int], thr: float) -> Tuple[int, float]:
    if direction in (0, None):
        move = (px - last_extreme) / last_extreme
        if abs(move) >= thr:
            return (1 if move > 0 else -1), px
        return 0, last_extreme
    if direction == 1:
        if px > last_extreme:
            return 1, px
        if (last_extreme - px)/last_extreme >= thr:
            return -1, px
        return 1, last_extreme
    # direction == -1
    if px < last_extreme:
        return -1, px
    if (px - last_extreme)/last_extreme >= thr:
        return 1, px
    return -1, last_extreme

def best_threshold(window: np.ndarray) -> float:
    best_thr, best_pnl = THR_GRID[0], -np.inf
    for thr in THR_GRID:
        dir_vec = np.zeros(len(window), np.int8)
        le, d = window[0], 0
        for k in range(1, len(window)):
            d, le = causal_update(window[k], le, d, thr)
            dir_vec[k] = d
        pnl = (dir_vec[:-1].astype(np.int32) *
               POSITION_LIMIT * np.diff(window)).sum()
        if pnl > best_pnl:
            best_pnl, best_thr = pnl, thr
    return best_thr

def realized_vol(returns: np.ndarray) -> float:
    return returns.std(ddof=0) * np.sqrt(252)

def sharpe_ratio(returns: np.ndarray) -> float:
    sigma = returns.std(ddof=0)
    return 0.0 if sigma==0 else (returns.mean()/sigma)*np.sqrt(252)

def skewness(returns: np.ndarray) -> float:
    m = returns.mean(); s = returns.std(ddof=0)
    return ((returns-m)**3).mean()/s**3 if s>0 else 0.0

# ──────────── 1. BasePIPTrader ────────────────────────────────────── #
class BasePIPTrader(Trader):
    class _State:
        __slots__ = ("n","w","next_cal","thr","dir","le")
        def __init__(self, n,w):
            self.n, self.w = n,w
            self.next_cal  = n
            self.thr = None
            self.dir = 0
            self.le  = None

    def __init__(self):
        super().__init__()
        self._states = [self._State(n,w) for n,w in zip(BEST_N,BEST_W)]
        print(f"{self.__class__.__name__} initialized.")

    @export
    def Alg(self, prc: np.ndarray) -> np.ndarray:
        N, t = prc.shape
        pos = np.zeros(N, np.int32)
        for i in range(N):
            if i in BLACK_LIST:
                continue
            st = self._states[i]
            px = prc[i, -1]
            if st.le is None:
                st.le = px
            if t >= st.next_cal:
                win = prc[i, t-st.n:t]
                st.thr = best_threshold(win)
                st.next_cal += st.w
                print(f"[t={t}] {self.__class__.__name__} inst {i} thr={st.thr:.4f}")
            if st.thr is None:
                continue
            st.dir, st.le = causal_update(px, st.le, st.dir, st.thr)
            raw = st.dir * POSITION_LIMIT
            scale = self.get_scale(i, prc, st)
            pos[i] = int(raw * scale)
        return pos

    def get_scale(self, i:int, prc:np.ndarray, st:_State) -> float:
        return 1.0

# ──────────── 2. ATRClampTrader ────────────────────────────────────── #
class ATRClampTrader(BasePIPTrader):
    def get_scale(self, i, prc, st):
        window = prc[i, -15:]
        tr     = np.abs(np.diff(window))
        atr14  = tr.mean() if len(tr)>0 else 1.0
        return float(np.clip(1.0/atr14, 0.5, 1.5))

# ──────────── 3. VolTargetTrader ──────────────────────────────────── #
class VolTargetTrader(BasePIPTrader):
    def __init__(self, target_vol=0.02):
        super().__init__(); self.tv = target_vol
    def get_scale(self, i, prc, st):
        win = prc[i, -st.n:]
        ret = np.diff(win)/win[:-1]
        vol = realized_vol(ret) if len(ret)>1 else 1.0
        return float(np.clip(self.tv/vol, 0.1, 5.0))

# ──────────── 4. SharpeScaledTrader ────────────────────────────────── #
class SharpeScaledTrader(BasePIPTrader):
    def get_scale(self, i, prc, st):
        win = prc[i, -st.n:]
        ret = np.diff(win)/win[:-1]
        sr  = sharpe_ratio(ret) if len(ret)>1 else 0.0
        return float(np.clip(sr, 0.0, 2.0))

# ──────────── 5. MomentumFilteredTrader ────────────────────────────── #
class MomentumFilteredTrader(BasePIPTrader):
    def get_scale(self, i, prc, st):
        p = prc[i]
        if len(p)<11 or st.dir==0:
            return 1.0
        sma_old = p[-11:-1].mean()
        sma_new = p[-10:].mean()
        return 1.0 if np.sign(sma_new-sma_old)==st.dir else 0.0

# ──────────── 6. CompositeScaledTrader ────────────────────────────── #
class CompositeScaledTrader(BasePIPTrader):
    def __init__(self, tv=0.02):
        super().__init__(); self.tv = tv
    def get_scale(self, i, prc, st):
        win = prc[i, -st.n:]
        ret = np.diff(win)/win[:-1]
        vol = realized_vol(ret) if len(ret)>1 else 1.0
        sr  = sharpe_ratio(ret) if len(ret)>1 else 0.0
        sc  = (self.tv/vol)*max(sr,0.0)
        return float(np.clip(sc, 0.1, 5.0))

# ──────────── 7. BetaScaledTrader ──────────────────────────────────── #
class BetaScaledTrader(BasePIPTrader):
    def get_scale(self, i, prc, st):
        win   = prc[i, -st.n:]
        ret_i = np.diff(win)/win[:-1]
        mkt   = prc[:, -st.n-1:-1]
        ret_m = np.diff(mkt)/mkt[:,:-1]
        mkt_ret = ret_m.mean(axis=0)
        cov = np.cov(ret_i, mkt_ret, ddof=0)[0,1]
        var = mkt_ret.var(ddof=0)
        beta= cov/var if var>0 else 1.0
        invb= 1.0/beta if beta!=0 else 1.0
        return float(np.clip(invb, 0.5,2.0))

# ──────────── 8. SkewScaledTrader ──────────────────────────────────── #
class SkewScaledTrader(BasePIPTrader):
    def get_scale(self, i, prc, st):
        win = prc[i, -st.n:]
        ret = np.diff(win)/win[:-1]
        sk  = skewness(ret) if len(ret)>1 else 0.0
        return float(np.clip(1.0+sk, 0.5,2.0))

# ──────────── 9. DrawdownStopTrader (fixed) ───────────────────────── #
class DrawdownStopTrader(BasePIPTrader):
    """
    Freeze instrument if drawdown >10% and unfreeze on recovery.
    """
    def __init__(self):
        super().__init__()
        N = len(self._states)
        self._peak   = [None]*N
        self._frozen = set()
        print("DrawdownStopTrader initialized.")

    @export
    def Alg(self, prc: np.ndarray) -> np.ndarray:
        pos = super().Alg(prc).astype(float)
        N, t = prc.shape
        for i in range(N):
            if i in BLACK_LIST:
                continue
            px = prc[i, -1]
            # init peak
            if self._peak[i] is None:
                self._peak[i] = px
            # update running peak
            self._peak[i] = max(self._peak[i], px)
            dd = (px - self._peak[i]) / self._peak[i]
            if i in self._frozen:
                if dd >= 0:
                    self._frozen.remove(i)
                else:
                    pos[i] = 0.0
            elif dd < -0.10:
                self._frozen.add(i)
                pos[i] = 0.0
        return pos.astype(int)

# ────────────10. MasterEnsembleTrader ──────────────────────────────── #
class MasterEnsembleTrader(Trader):
    def __init__(self):
        super().__init__()
        self.mods = [
            BasePIPTrader(), ATRClampTrader(), VolTargetTrader(),
            SharpeScaledTrader(), MomentumFilteredTrader(),
            CompositeScaledTrader(), BetaScaledTrader(), SkewScaledTrader(),
            DrawdownStopTrader()
        ]
        print("MasterEnsembleTrader ready.")

    @export
    def Alg(self, prc: np.ndarray) -> np.ndarray:
        N = prc.shape[0]
        agg = np.zeros(N, dtype=float)
        for m in self.mods:
            agg += m.Alg(prc)
        agg /= len(self.mods)
        pos = np.zeros(N, dtype=int)
        for i in range(N):
            if i in BLACK_LIST: continue
            if agg[i] > 0: pos[i] = POSITION_LIMIT
            elif agg[i] < 0: pos[i] = -POSITION_LIMIT
        return pos

# ─────────────────── Backtest & summary ───────────────────────────── #
def load_prices(path="prices.txt") -> np.ndarray:
    raw = np.loadtxt(path)
    if raw.ndim!=2 or raw.shape[1]!=50:
        raise ValueError(f"Expected 1000×50 data, got {raw.shape}")
    return raw.T

def backtest(trader: Trader, px: np.ndarray) -> Tuple[np.ndarray,np.ndarray]:
    n, T = px.shape
    cum   = np.zeros(T)
    daily = np.zeros(T)
    prev  = np.zeros(n)
    for t in range(1, T):
        pos = trader.Alg(px[:,:t])
        dpx = px[:,t] - px[:,t-1]
        gross = np.dot(prev, dpx)
        trades= pos - prev
        tv    = np.dot(np.abs(trades), px[:,t])
        comm  = COMMISSION_RATE * tv
        pnl   = gross - comm
        cum[t]   = cum[t-1] + pnl
        daily[t] = pnl
        prev     = pos
    return cum, daily

def perf_summary(name, cum, daily):
    mean, std = daily.mean(), daily.std(ddof=1)
    sharpe    = (mean/std)*np.sqrt(252) if std>0 else 0.0
    wr        = np.mean(daily>0)*100
    print(f"{name:20s} | Final PnL: {cum[-1]:9.0f} | Sharpe: {sharpe:5.2f} | Win%: {wr:5.1f}")

if __name__=="__main__":
    prices = load_prices()
    print(f"Loaded {prices.shape[0]} instruments × {prices.shape[1]} bars\n")

    traders = {
        "BasePIP":        BasePIPTrader(),
        "ATRClamp":       ATRClampTrader(),
        "VolTarget":      VolTargetTrader(),
        "SharpeScaled":   SharpeScaledTrader(),
        "MomentumFilter": MomentumFilteredTrader(),
        "Composite":      CompositeScaledTrader(),
        "BetaScaled":     BetaScaledTrader(),
        "SkewScaled":     SkewScaledTrader(),
        "DrawdownStop":   DrawdownStopTrader(),
        "MasterEnsemble": MasterEnsembleTrader(),
    }

    results, dailies = {}, {}
    for name, tr in traders.items():
        print(f"Running {name}…")
        cum, daily = backtest(tr, prices)
        results[name], dailies[name] = cum, daily

    print("\nPerformance Summary:")
    for name in traders:
        perf_summary(name, results[name], dailies[name])

    plt.figure(figsize=(12,6))
    for name, cum in results.items():
        plt.plot(cum, label=name)
    plt.title("Equity Curves: 10 PIP‑Style Traders (Fixed)")
    plt.xlabel("Bar")
    plt.ylabel("Cumulative PnL")
    plt.legend(ncol=2, fontsize="small")
    plt.tight_layout()
    plt.show()
